In [1]:
import opendssdirect as dss
from typing import List
import math
import json


#you don't have to create busbars, they are already defined in other element
#def create_other_elements(in_data,x):
#    print("cos")   

in_data_pandapower = {
    '0': {'typ': 'PowerFlowPandaPower Parameters', 'frequency': '50', 'algorithm': 'nr', 'calculate_voltage_angles': 'auto', 'initialization': 'auto', 'exportPython': False, 'exportPandapowerResults': False, 'run_control': False, 'user_email': 'maciej@gmail.com'}, '1': {'name': 'mxCell_141', 'id': 'aSc5nKb4JZ8KHuJaPj6X-2', 'userFriendlyName': 'External Grid', 'bus': 'mxCell_145', 'typ': 'External Grid0', 'vm_pu': '1', 'va_degree': '0', 's_sc_max_mva': '100000000', 's_sc_min_mva': '0', 'rx_max': '0', 'rx_min': '0', 'r0x0_max': '0', 'x0x_max': '0', 'in_service': 'true'}, '2': {'name': 'mxCell_153', 'id': 'aSc5nKb4JZ8KHuJaPj6X-12', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_197', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '3': {'name': 'mxCell_202', 'id': 'aSc5nKb4JZ8KHuJaPj6X-109', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_209', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '4': {'name': 'mxCell_222', 'id': 'aSc5nKb4JZ8KHuJaPj6X-137', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_229', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '5': {'name': 'mxCell_238', 'id': 'aSc5nKb4JZ8KHuJaPj6X-153', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_245', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '6': {'name': 'mxCell_254', 'id': 'qValLhmmgod3405-91VG-2', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_261', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '7': {'name': 'mxCell_270', 'id': 'qValLhmmgod3405-91VG-215', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_275', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '8': {'name': 'mxCell_280', 'id': 'qValLhmmgod3405-91VG-226', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_287', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '9': {'name': 'mxCell_300', 'id': 'qValLhmmgod3405-91VG-246', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_307', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '10': {'name': 'mxCell_316', 'id': 'qValLhmmgod3405-91VG-262', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_323', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '11': {'name': 'mxCell_332', 'id': 'qValLhmmgod3405-91VG-278', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_339', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '12': {'name': 'mxCell_358', 'id': 'qValLhmmgod3405-91VG-316', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_363', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '13': {'name': 'mxCell_368', 'id': 'qValLhmmgod3405-91VG-326', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_375', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '14': {'name': 'mxCell_388', 'id': 'qValLhmmgod3405-91VG-346', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_395', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '15': {'name': 'mxCell_404', 'id': 'qValLhmmgod3405-91VG-362', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_411', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '16': {'name': 'mxCell_420', 'id': 'qValLhmmgod3405-91VG-378', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_427', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '17': {'name': 'mxCell_436', 'id': 'qValLhmmgod3405-91VG-555', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_441', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '18': {'name': 'mxCell_446', 'id': 'qValLhmmgod3405-91VG-566', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_453', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '19': {'name': 'mxCell_466', 'id': 'qValLhmmgod3405-91VG-586', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_473', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '20': {'name': 'mxCell_482', 'id': 'qValLhmmgod3405-91VG-602', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_489', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '21': {'name': 'mxCell_498', 'id': 'qValLhmmgod3405-91VG-618', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_505', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '22': {'name': 'mxCell_514', 'id': 'qValLhmmgod3405-91VG-634', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_519', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '23': {'name': 'mxCell_524', 'id': 'qValLhmmgod3405-91VG-644', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_531', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '24': {'name': 'mxCell_544', 'id': 'qValLhmmgod3405-91VG-664', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_551', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '25': {'name': 'mxCell_560', 'id': 'qValLhmmgod3405-91VG-680', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_567', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '26': {'name': 'mxCell_576', 'id': 'qValLhmmgod3405-91VG-696', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_583', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '27': {'name': 'mxCell_592', 'id': 'qValLhmmgod3405-91VG-712', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_597', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '28': {'name': 'mxCell_602', 'id': 'qValLhmmgod3405-91VG-722', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_609', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '29': {'name': 'mxCell_622', 'id': 'qValLhmmgod3405-91VG-742', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_629', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '30': {'name': 'mxCell_638', 'id': 'qValLhmmgod3405-91VG-758', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_645', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '31': {'name': 'mxCell_654', 'id': 'qValLhmmgod3405-91VG-774', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_661', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '32': {'name': 'mxCell_685', 'id': 'qValLhmmgod3405-91VG-816', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_728', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '33': {'name': 'mxCell_733', 'id': 'qValLhmmgod3405-91VG-865', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_740', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '34': {'name': 'mxCell_753', 'id': 'qValLhmmgod3405-91VG-885', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_760', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '35': {'name': 'mxCell_769', 'id': 'qValLhmmgod3405-91VG-901', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_776', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '36': {'name': 'mxCell_785', 'id': 'qValLhmmgod3405-91VG-917', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_792', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '37': {'name': 'mxCell_801', 'id': 'qValLhmmgod3405-91VG-933', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_806', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '38': {'name': 'mxCell_811', 'id': 'qValLhmmgod3405-91VG-943', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_818', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '39': {'name': 'mxCell_831', 'id': 'qValLhmmgod3405-91VG-963', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_838', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '40': {'name': 'mxCell_847', 'id': 'qValLhmmgod3405-91VG-979', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_854', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '41': {'name': 'mxCell_863', 'id': 'qValLhmmgod3405-91VG-995', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_870', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '42': {'name': 'mxCell_889', 'id': 'qValLhmmgod3405-91VG-1021', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_894', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '43': {'name': 'mxCell_899', 'id': 'qValLhmmgod3405-91VG-1031', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_906', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '44': {'name': 'mxCell_919', 'id': 'qValLhmmgod3405-91VG-1051', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_926', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '45': {'name': 'mxCell_935', 'id': 'qValLhmmgod3405-91VG-1067', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_942', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '46': {'name': 'mxCell_951', 'id': 'qValLhmmgod3405-91VG-1083', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_958', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '47': {'name': 'mxCell_967', 'id': 'qValLhmmgod3405-91VG-1099', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_972', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '48': {'name': 'mxCell_977', 'id': 'qValLhmmgod3405-91VG-1109', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_984', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '49': {'name': 'mxCell_997', 'id': 'qValLhmmgod3405-91VG-1129', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1004', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '50': {'name': 'mxCell_1013', 'id': 'qValLhmmgod3405-91VG-1145', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1020', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '51': {'name': 'mxCell_1029', 'id': 'qValLhmmgod3405-91VG-1161', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1036', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '52': {'name': 'mxCell_1045', 'id': 'qValLhmmgod3405-91VG-1177', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1050', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '53': {'name': 'mxCell_1055', 'id': 'qValLhmmgod3405-91VG-1187', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1062', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '54': {'name': 'mxCell_1075', 'id': 'qValLhmmgod3405-91VG-1207', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1082', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '55': {'name': 'mxCell_1091', 'id': 'qValLhmmgod3405-91VG-1223', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1098', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '56': {'name': 'mxCell_1107', 'id': 'qValLhmmgod3405-91VG-1239', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1114', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '57': {'name': 'mxCell_1123', 'id': 'qValLhmmgod3405-91VG-1255', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1128', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '58': {'name': 'mxCell_1133', 'id': 'qValLhmmgod3405-91VG-1265', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1140', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '59': {'name': 'mxCell_1153', 'id': 'qValLhmmgod3405-91VG-1285', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1160', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '60': {'name': 'mxCell_1169', 'id': 'qValLhmmgod3405-91VG-1301', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1176', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '61': {'name': 'mxCell_1185', 'id': 'qValLhmmgod3405-91VG-1317', 'userFriendlyName': 'Static Generator', 'bus': 'mxCell_1192', 'typ': 'Static Generator', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': 'true'}, '62': {'typ': 'Bus0', 'name': 'mxCell_145', 'id': 'aSc5nKb4JZ8KHuJaPj6X-6', 'vn_kv': '400', 'userFriendlyName': 'Bus'}, '63': {'typ': 'Bus1', 'name': 'mxCell_148', 'id': 'aSc5nKb4JZ8KHuJaPj6X-9', 'vn_kv': '400', 'userFriendlyName': 'Bus'}, '64': {'typ': 'Bus2', 'name': 'mxCell_157', 'id': 'aSc5nKb4JZ8KHuJaPj6X-16', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '65': {'typ': 'Bus3', 'name': 'mxCell_162', 'id': 'aSc5nKb4JZ8KHuJaPj6X-25', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '66': {'typ': 'Bus4', 'name': 'mxCell_165', 'id': 'aSc5nKb4JZ8KHuJaPj6X-30', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '67': {'typ': 'Bus5', 'name': 'mxCell_181', 'id': 'aSc5nKb4JZ8KHuJaPj6X-36', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '68': {'typ': 'Bus6', 'name': 'mxCell_190', 'id': 'aSc5nKb4JZ8KHuJaPj6X-52', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '69': {'typ': 'Bus7', 'name': 'mxCell_194', 'id': 'aSc5nKb4JZ8KHuJaPj6X-58', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '70': {'typ': 'Bus8', 'name': 'mxCell_197', 'id': 'aSc5nKb4JZ8KHuJaPj6X-64', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '71': {'typ': 'Bus9', 'name': 'mxCell_206', 'id': 'aSc5nKb4JZ8KHuJaPj6X-113', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '72': {'typ': 'Bus10', 'name': 'mxCell_209', 'id': 'aSc5nKb4JZ8KHuJaPj6X-116', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '73': {'typ': 'Bus11', 'name': 'mxCell_226', 'id': 'aSc5nKb4JZ8KHuJaPj6X-141', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '74': {'typ': 'Bus12', 'name': 'mxCell_229', 'id': 'aSc5nKb4JZ8KHuJaPj6X-144', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '75': {'typ': 'Bus13', 'name': 'mxCell_242', 'id': 'aSc5nKb4JZ8KHuJaPj6X-157', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '76': {'typ': 'Bus14', 'name': 'mxCell_245', 'id': 'aSc5nKb4JZ8KHuJaPj6X-160', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '77': {'typ': 'Bus15', 'name': 'mxCell_258', 'id': 'qValLhmmgod3405-91VG-6', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '78': {'typ': 'Bus16', 'name': 'mxCell_261', 'id': 'qValLhmmgod3405-91VG-9', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '79': {'typ': 'Bus17', 'name': 'mxCell_272', 'id': 'qValLhmmgod3405-91VG-218', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '80': {'typ': 'Bus18', 'name': 'mxCell_275', 'id': 'qValLhmmgod3405-91VG-221', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '81': {'typ': 'Bus19', 'name': 'mxCell_284', 'id': 'qValLhmmgod3405-91VG-230', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '82': {'typ': 'Bus20', 'name': 'mxCell_287', 'id': 'qValLhmmgod3405-91VG-233', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '83': {'typ': 'Bus21', 'name': 'mxCell_304', 'id': 'qValLhmmgod3405-91VG-250', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '84': {'typ': 'Bus22', 'name': 'mxCell_307', 'id': 'qValLhmmgod3405-91VG-253', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '85': {'typ': 'Bus23', 'name': 'mxCell_320', 'id': 'qValLhmmgod3405-91VG-266', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '86': {'typ': 'Bus24', 'name': 'mxCell_323', 'id': 'qValLhmmgod3405-91VG-269', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '87': {'typ': 'Bus25', 'name': 'mxCell_336', 'id': 'qValLhmmgod3405-91VG-282', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '88': {'typ': 'Bus26', 'name': 'mxCell_339', 'id': 'qValLhmmgod3405-91VG-285', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '89': {'typ': 'Bus27', 'name': 'mxCell_360', 'id': 'qValLhmmgod3405-91VG-318', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '90': {'typ': 'Bus28', 'name': 'mxCell_363', 'id': 'qValLhmmgod3405-91VG-321', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '91': {'typ': 'Bus29', 'name': 'mxCell_372', 'id': 'qValLhmmgod3405-91VG-330', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '92': {'typ': 'Bus30', 'name': 'mxCell_375', 'id': 'qValLhmmgod3405-91VG-333', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '93': {'typ': 'Bus31', 'name': 'mxCell_392', 'id': 'qValLhmmgod3405-91VG-350', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '94': {'typ': 'Bus32', 'name': 'mxCell_395', 'id': 'qValLhmmgod3405-91VG-353', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '95': {'typ': 'Bus33', 'name': 'mxCell_408', 'id': 'qValLhmmgod3405-91VG-366', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '96': {'typ': 'Bus34', 'name': 'mxCell_411', 'id': 'qValLhmmgod3405-91VG-369', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '97': {'typ': 'Bus35', 'name': 'mxCell_424', 'id': 'qValLhmmgod3405-91VG-382', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '98': {'typ': 'Bus36', 'name': 'mxCell_427', 'id': 'qValLhmmgod3405-91VG-385', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '99': {'typ': 'Bus37', 'name': 'mxCell_438', 'id': 'qValLhmmgod3405-91VG-558', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '100': {'typ': 'Bus38', 'name': 'mxCell_441', 'id': 'qValLhmmgod3405-91VG-561', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '101': {'typ': 'Bus39', 'name': 'mxCell_450', 'id': 'qValLhmmgod3405-91VG-570', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '102': {'typ': 'Bus40', 'name': 'mxCell_453', 'id': 'qValLhmmgod3405-91VG-573', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '103': {'typ': 'Bus41', 'name': 'mxCell_470', 'id': 'qValLhmmgod3405-91VG-590', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '104': {'typ': 'Bus42', 'name': 'mxCell_473', 'id': 'qValLhmmgod3405-91VG-593', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '105': {'typ': 'Bus43', 'name': 'mxCell_486', 'id': 'qValLhmmgod3405-91VG-606', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '106': {'typ': 'Bus44', 'name': 'mxCell_489', 'id': 'qValLhmmgod3405-91VG-609', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '107': {'typ': 'Bus45', 'name': 'mxCell_502', 'id': 'qValLhmmgod3405-91VG-622', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '108': {'typ': 'Bus46', 'name': 'mxCell_505', 'id': 'qValLhmmgod3405-91VG-625', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '109': {'typ': 'Bus47', 'name': 'mxCell_516', 'id': 'qValLhmmgod3405-91VG-636', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '110': {'typ': 'Bus48', 'name': 'mxCell_519', 'id': 'qValLhmmgod3405-91VG-639', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '111': {'typ': 'Bus49', 'name': 'mxCell_528', 'id': 'qValLhmmgod3405-91VG-648', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '112': {'typ': 'Bus50', 'name': 'mxCell_531', 'id': 'qValLhmmgod3405-91VG-651', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '113': {'typ': 'Bus51', 'name': 'mxCell_548', 'id': 'qValLhmmgod3405-91VG-668', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '114': {'typ': 'Bus52', 'name': 'mxCell_551', 'id': 'qValLhmmgod3405-91VG-671', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '115': {'typ': 'Bus53', 'name': 'mxCell_564', 'id': 'qValLhmmgod3405-91VG-684', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '116': {'typ': 'Bus54', 'name': 'mxCell_567', 'id': 'qValLhmmgod3405-91VG-687', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '117': {'typ': 'Bus55', 'name': 'mxCell_580', 'id': 'qValLhmmgod3405-91VG-700', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '118': {'typ': 'Bus56', 'name': 'mxCell_583', 'id': 'qValLhmmgod3405-91VG-703', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '119': {'typ': 'Bus57', 'name': 'mxCell_594', 'id': 'qValLhmmgod3405-91VG-714', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '120': {'typ': 'Bus58', 'name': 'mxCell_597', 'id': 'qValLhmmgod3405-91VG-717', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '121': {'typ': 'Bus59', 'name': 'mxCell_606', 'id': 'qValLhmmgod3405-91VG-726', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '122': {'typ': 'Bus60', 'name': 'mxCell_609', 'id': 'qValLhmmgod3405-91VG-729', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '123': {'typ': 'Bus61', 'name': 'mxCell_626', 'id': 'qValLhmmgod3405-91VG-746', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '124': {'typ': 'Bus62', 'name': 'mxCell_629', 'id': 'qValLhmmgod3405-91VG-749', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '125': {'typ': 'Bus63', 'name': 'mxCell_642', 'id': 'qValLhmmgod3405-91VG-762', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '126': {'typ': 'Bus64', 'name': 'mxCell_645', 'id': 'qValLhmmgod3405-91VG-765', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '127': {'typ': 'Bus65', 'name': 'mxCell_658', 'id': 'qValLhmmgod3405-91VG-778', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '128': {'typ': 'Bus66', 'name': 'mxCell_661', 'id': 'qValLhmmgod3405-91VG-781', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '129': {'typ': 'Bus67', 'name': 'mxCell_689', 'id': 'qValLhmmgod3405-91VG-820', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '130': {'typ': 'Bus68', 'name': 'mxCell_694', 'id': 'qValLhmmgod3405-91VG-825', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '131': {'typ': 'Bus69', 'name': 'mxCell_697', 'id': 'qValLhmmgod3405-91VG-828', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '132': {'typ': 'Bus70', 'name': 'mxCell_713', 'id': 'qValLhmmgod3405-91VG-844', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '133': {'typ': 'Bus71', 'name': 'mxCell_721', 'id': 'qValLhmmgod3405-91VG-853', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '134': {'typ': 'Bus72', 'name': 'mxCell_725', 'id': 'qValLhmmgod3405-91VG-857', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '135': {'typ': 'Bus73', 'name': 'mxCell_728', 'id': 'qValLhmmgod3405-91VG-860', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '136': {'typ': 'Bus74', 'name': 'mxCell_737', 'id': 'qValLhmmgod3405-91VG-869', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '137': {'typ': 'Bus75', 'name': 'mxCell_740', 'id': 'qValLhmmgod3405-91VG-872', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '138': {'typ': 'Bus76', 'name': 'mxCell_757', 'id': 'qValLhmmgod3405-91VG-889', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '139': {'typ': 'Bus77', 'name': 'mxCell_760', 'id': 'qValLhmmgod3405-91VG-892', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '140': {'typ': 'Bus78', 'name': 'mxCell_773', 'id': 'qValLhmmgod3405-91VG-905', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '141': {'typ': 'Bus79', 'name': 'mxCell_776', 'id': 'qValLhmmgod3405-91VG-908', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '142': {'typ': 'Bus80', 'name': 'mxCell_789', 'id': 'qValLhmmgod3405-91VG-921', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '143': {'typ': 'Bus81', 'name': 'mxCell_792', 'id': 'qValLhmmgod3405-91VG-924', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '144': {'typ': 'Bus82', 'name': 'mxCell_803', 'id': 'qValLhmmgod3405-91VG-935', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '145': {'typ': 'Bus83', 'name': 'mxCell_806', 'id': 'qValLhmmgod3405-91VG-938', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '146': {'typ': 'Bus84', 'name': 'mxCell_815', 'id': 'qValLhmmgod3405-91VG-947', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '147': {'typ': 'Bus85', 'name': 'mxCell_818', 'id': 'qValLhmmgod3405-91VG-950', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '148': {'typ': 'Bus86', 'name': 'mxCell_835', 'id': 'qValLhmmgod3405-91VG-967', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '149': {'typ': 'Bus87', 'name': 'mxCell_838', 'id': 'qValLhmmgod3405-91VG-970', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '150': {'typ': 'Bus88', 'name': 'mxCell_851', 'id': 'qValLhmmgod3405-91VG-983', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '151': {'typ': 'Bus89', 'name': 'mxCell_854', 'id': 'qValLhmmgod3405-91VG-986', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '152': {'typ': 'Bus90', 'name': 'mxCell_867', 'id': 'qValLhmmgod3405-91VG-999', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '153': {'typ': 'Bus91', 'name': 'mxCell_870', 'id': 'qValLhmmgod3405-91VG-1002', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '154': {'typ': 'Bus92', 'name': 'mxCell_891', 'id': 'qValLhmmgod3405-91VG-1023', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '155': {'typ': 'Bus93', 'name': 'mxCell_894', 'id': 'qValLhmmgod3405-91VG-1026', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '156': {'typ': 'Bus94', 'name': 'mxCell_903', 'id': 'qValLhmmgod3405-91VG-1035', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '157': {'typ': 'Bus95', 'name': 'mxCell_906', 'id': 'qValLhmmgod3405-91VG-1038', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '158': {'typ': 'Bus96', 'name': 'mxCell_923', 'id': 'qValLhmmgod3405-91VG-1055', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '159': {'typ': 'Bus97', 'name': 'mxCell_926', 'id': 'qValLhmmgod3405-91VG-1058', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '160': {'typ': 'Bus98', 'name': 'mxCell_939', 'id': 'qValLhmmgod3405-91VG-1071', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '161': {'typ': 'Bus99', 'name': 'mxCell_942', 'id': 'qValLhmmgod3405-91VG-1074', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '162': {'typ': 'Bus100', 'name': 'mxCell_955', 'id': 'qValLhmmgod3405-91VG-1087', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '163': {'typ': 'Bus101', 'name': 'mxCell_958', 'id': 'qValLhmmgod3405-91VG-1090', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '164': {'typ': 'Bus102', 'name': 'mxCell_969', 'id': 'qValLhmmgod3405-91VG-1101', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '165': {'typ': 'Bus103', 'name': 'mxCell_972', 'id': 'qValLhmmgod3405-91VG-1104', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '166': {'typ': 'Bus104', 'name': 'mxCell_981', 'id': 'qValLhmmgod3405-91VG-1113', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '167': {'typ': 'Bus105', 'name': 'mxCell_984', 'id': 'qValLhmmgod3405-91VG-1116', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '168': {'typ': 'Bus106', 'name': 'mxCell_1001', 'id': 'qValLhmmgod3405-91VG-1133', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '169': {'typ': 'Bus107', 'name': 'mxCell_1004', 'id': 'qValLhmmgod3405-91VG-1136', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '170': {'typ': 'Bus108', 'name': 'mxCell_1017', 'id': 'qValLhmmgod3405-91VG-1149', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '171': {'typ': 'Bus109', 'name': 'mxCell_1020', 'id': 'qValLhmmgod3405-91VG-1152', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '172': {'typ': 'Bus110', 'name': 'mxCell_1033', 'id': 'qValLhmmgod3405-91VG-1165', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '173': {'typ': 'Bus111', 'name': 'mxCell_1036', 'id': 'qValLhmmgod3405-91VG-1168', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '174': {'typ': 'Bus112', 'name': 'mxCell_1047', 'id': 'qValLhmmgod3405-91VG-1179', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '175': {'typ': 'Bus113', 'name': 'mxCell_1050', 'id': 'qValLhmmgod3405-91VG-1182', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '176': {'typ': 'Bus114', 'name': 'mxCell_1059', 'id': 'qValLhmmgod3405-91VG-1191', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '177': {'typ': 'Bus115', 'name': 'mxCell_1062', 'id': 'qValLhmmgod3405-91VG-1194', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '178': {'typ': 'Bus116', 'name': 'mxCell_1079', 'id': 'qValLhmmgod3405-91VG-1211', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '179': {'typ': 'Bus117', 'name': 'mxCell_1082', 'id': 'qValLhmmgod3405-91VG-1214', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '180': {'typ': 'Bus118', 'name': 'mxCell_1095', 'id': 'qValLhmmgod3405-91VG-1227', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '181': {'typ': 'Bus119', 'name': 'mxCell_1098', 'id': 'qValLhmmgod3405-91VG-1230', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '182': {'typ': 'Bus120', 'name': 'mxCell_1111', 'id': 'qValLhmmgod3405-91VG-1243', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '183': {'typ': 'Bus121', 'name': 'mxCell_1114', 'id': 'qValLhmmgod3405-91VG-1246', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '184': {'typ': 'Bus122', 'name': 'mxCell_1125', 'id': 'qValLhmmgod3405-91VG-1257', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '185': {'typ': 'Bus123', 'name': 'mxCell_1128', 'id': 'qValLhmmgod3405-91VG-1260', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '186': {'typ': 'Bus124', 'name': 'mxCell_1137', 'id': 'qValLhmmgod3405-91VG-1269', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '187': {'typ': 'Bus125', 'name': 'mxCell_1140', 'id': 'qValLhmmgod3405-91VG-1272', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '188': {'typ': 'Bus126', 'name': 'mxCell_1157', 'id': 'qValLhmmgod3405-91VG-1289', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '189': {'typ': 'Bus127', 'name': 'mxCell_1160', 'id': 'qValLhmmgod3405-91VG-1292', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '190': {'typ': 'Bus128', 'name': 'mxCell_1173', 'id': 'qValLhmmgod3405-91VG-1305', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '191': {'typ': 'Bus129', 'name': 'mxCell_1176', 'id': 'qValLhmmgod3405-91VG-1308', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '192': {'typ': 'Bus130', 'name': 'mxCell_1189', 'id': 'qValLhmmgod3405-91VG-1321', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '193': {'typ': 'Bus131', 'name': 'mxCell_1192', 'id': 'qValLhmmgod3405-91VG-1324', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '194': {'typ': 'Transformer0', 'name': 'mxCell_215', 'id': 'aSc5nKb4JZ8KHuJaPj6X-123', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_194', 'lv_bus': 'mxCell_197', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '195': {'typ': 'Transformer1', 'name': 'mxCell_219', 'id': 'aSc5nKb4JZ8KHuJaPj6X-128', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_206', 'lv_bus': 'mxCell_209', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '196': {'typ': 'Transformer2', 'name': 'mxCell_235', 'id': 'aSc5nKb4JZ8KHuJaPj6X-150', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_226', 'lv_bus': 'mxCell_229', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '197': {'typ': 'Transformer3', 'name': 'mxCell_251', 'id': 'aSc5nKb4JZ8KHuJaPj6X-166', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_242', 'lv_bus': 'mxCell_245', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '198': {'typ': 'Transformer4', 'name': 'mxCell_267', 'id': 'qValLhmmgod3405-91VG-15', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_258', 'lv_bus': 'mxCell_261', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '199': {'typ': 'Transformer5', 'name': 'mxCell_293', 'id': 'qValLhmmgod3405-91VG-239', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_272', 'lv_bus': 'mxCell_275', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '200': {'typ': 'Transformer6', 'name': 'mxCell_297', 'id': 'qValLhmmgod3405-91VG-243', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_284', 'lv_bus': 'mxCell_287', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '201': {'typ': 'Transformer7', 'name': 'mxCell_313', 'id': 'qValLhmmgod3405-91VG-259', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_304', 'lv_bus': 'mxCell_307', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '202': {'typ': 'Transformer8', 'name': 'mxCell_329', 'id': 'qValLhmmgod3405-91VG-275', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_320', 'lv_bus': 'mxCell_323', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '203': {'typ': 'Transformer9', 'name': 'mxCell_345', 'id': 'qValLhmmgod3405-91VG-291', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_336', 'lv_bus': 'mxCell_339', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '204': {'typ': 'Transformer10', 'name': 'mxCell_349', 'id': 'qValLhmmgod3405-91VG-294', 'userFriendlyName': '300MVA 400/275kV', 'hv_bus': 'mxCell_148', 'lv_bus': 'mxCell_157', 'sn_mva': '300', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '205': {'typ': 'Transformer11', 'name': 'mxCell_353', 'id': 'qValLhmmgod3405-91VG-306', 'userFriendlyName': '300MVA 275/66kV', 'hv_bus': 'mxCell_165', 'lv_bus': 'mxCell_181', 'sn_mva': '300', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '206': {'typ': 'Transformer12', 'name': 'mxCell_381', 'id': 'qValLhmmgod3405-91VG-339', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_360', 'lv_bus': 'mxCell_363', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '207': {'typ': 'Transformer13', 'name': 'mxCell_385', 'id': 'qValLhmmgod3405-91VG-343', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_372', 'lv_bus': 'mxCell_375', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '208': {'typ': 'Transformer14', 'name': 'mxCell_401', 'id': 'qValLhmmgod3405-91VG-359', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_392', 'lv_bus': 'mxCell_395', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '209': {'typ': 'Transformer15', 'name': 'mxCell_417', 'id': 'qValLhmmgod3405-91VG-375', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_408', 'lv_bus': 'mxCell_411', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '210': {'typ': 'Transformer16', 'name': 'mxCell_433', 'id': 'qValLhmmgod3405-91VG-391', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_424', 'lv_bus': 'mxCell_427', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '211': {'typ': 'Transformer17', 'name': 'mxCell_459', 'id': 'qValLhmmgod3405-91VG-579', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_438', 'lv_bus': 'mxCell_441', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '212': {'typ': 'Transformer18', 'name': 'mxCell_463', 'id': 'qValLhmmgod3405-91VG-583', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_450', 'lv_bus': 'mxCell_453', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '213': {'typ': 'Transformer19', 'name': 'mxCell_479', 'id': 'qValLhmmgod3405-91VG-599', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_470', 'lv_bus': 'mxCell_473', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '214': {'typ': 'Transformer20', 'name': 'mxCell_495', 'id': 'qValLhmmgod3405-91VG-615', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_486', 'lv_bus': 'mxCell_489', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '215': {'typ': 'Transformer21', 'name': 'mxCell_511', 'id': 'qValLhmmgod3405-91VG-631', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_502', 'lv_bus': 'mxCell_505', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '216': {'typ': 'Transformer22', 'name': 'mxCell_537', 'id': 'qValLhmmgod3405-91VG-657', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_516', 'lv_bus': 'mxCell_519', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '217': {'typ': 'Transformer23', 'name': 'mxCell_541', 'id': 'qValLhmmgod3405-91VG-661', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_528', 'lv_bus': 'mxCell_531', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '218': {'typ': 'Transformer24', 'name': 'mxCell_557', 'id': 'qValLhmmgod3405-91VG-677', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_548', 'lv_bus': 'mxCell_551', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '219': {'typ': 'Transformer25', 'name': 'mxCell_573', 'id': 'qValLhmmgod3405-91VG-693', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_564', 'lv_bus': 'mxCell_567', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '220': {'typ': 'Transformer26', 'name': 'mxCell_589', 'id': 'qValLhmmgod3405-91VG-709', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_580', 'lv_bus': 'mxCell_583', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '221': {'typ': 'Transformer27', 'name': 'mxCell_615', 'id': 'qValLhmmgod3405-91VG-735', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_594', 'lv_bus': 'mxCell_597', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '222': {'typ': 'Transformer28', 'name': 'mxCell_619', 'id': 'qValLhmmgod3405-91VG-739', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_606', 'lv_bus': 'mxCell_609', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '223': {'typ': 'Transformer29', 'name': 'mxCell_635', 'id': 'qValLhmmgod3405-91VG-755', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_626', 'lv_bus': 'mxCell_629', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '224': {'typ': 'Transformer30', 'name': 'mxCell_651', 'id': 'qValLhmmgod3405-91VG-771', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_642', 'lv_bus': 'mxCell_645', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '225': {'typ': 'Transformer31', 'name': 'mxCell_667', 'id': 'qValLhmmgod3405-91VG-787', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_658', 'lv_bus': 'mxCell_661', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '226': {'typ': 'Transformer32', 'name': 'mxCell_671', 'id': 'qValLhmmgod3405-91VG-794', 'userFriendlyName': '300MVA 275/66kV', 'hv_bus': 'mxCell_165', 'lv_bus': 'mxCell_181', 'sn_mva': '300', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '227': {'typ': 'Transformer33', 'name': 'mxCell_675', 'id': 'qValLhmmgod3405-91VG-798', 'userFriendlyName': '300MVA 400/275kV', 'hv_bus': 'mxCell_148', 'lv_bus': 'mxCell_157', 'sn_mva': '300', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '228': {'typ': 'Transformer34', 'name': 'mxCell_746', 'id': 'qValLhmmgod3405-91VG-878', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_725', 'lv_bus': 'mxCell_728', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '229': {'typ': 'Transformer35', 'name': 'mxCell_750', 'id': 'qValLhmmgod3405-91VG-882', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_737', 'lv_bus': 'mxCell_740', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '230': {'typ': 'Transformer36', 'name': 'mxCell_766', 'id': 'qValLhmmgod3405-91VG-898', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_757', 'lv_bus': 'mxCell_760', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '231': {'typ': 'Transformer37', 'name': 'mxCell_782', 'id': 'qValLhmmgod3405-91VG-914', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_773', 'lv_bus': 'mxCell_776', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '232': {'typ': 'Transformer38', 'name': 'mxCell_798', 'id': 'qValLhmmgod3405-91VG-930', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_789', 'lv_bus': 'mxCell_792', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '233': {'typ': 'Transformer39', 'name': 'mxCell_824', 'id': 'qValLhmmgod3405-91VG-956', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_803', 'lv_bus': 'mxCell_806', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '234': {'typ': 'Transformer40', 'name': 'mxCell_828', 'id': 'qValLhmmgod3405-91VG-960', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_815', 'lv_bus': 'mxCell_818', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '235': {'typ': 'Transformer41', 'name': 'mxCell_844', 'id': 'qValLhmmgod3405-91VG-976', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_835', 'lv_bus': 'mxCell_838', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '236': {'typ': 'Transformer42', 'name': 'mxCell_860', 'id': 'qValLhmmgod3405-91VG-992', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_851', 'lv_bus': 'mxCell_854', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '237': {'typ': 'Transformer43', 'name': 'mxCell_876', 'id': 'qValLhmmgod3405-91VG-1008', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_867', 'lv_bus': 'mxCell_870', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '238': {'typ': 'Transformer44', 'name': 'mxCell_880', 'id': 'qValLhmmgod3405-91VG-1012', 'userFriendlyName': '300MVA 400/275kV', 'hv_bus': 'mxCell_148', 'lv_bus': 'mxCell_689', 'sn_mva': '300', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '239': {'typ': 'Transformer45', 'name': 'mxCell_884', 'id': 'qValLhmmgod3405-91VG-1016', 'userFriendlyName': '300MVA 275/66kV', 'hv_bus': 'mxCell_697', 'lv_bus': 'mxCell_713', 'sn_mva': '300', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '240': {'typ': 'Transformer46', 'name': 'mxCell_912', 'id': 'qValLhmmgod3405-91VG-1044', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_891', 'lv_bus': 'mxCell_894', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '241': {'typ': 'Transformer47', 'name': 'mxCell_916', 'id': 'qValLhmmgod3405-91VG-1048', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_903', 'lv_bus': 'mxCell_906', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '242': {'typ': 'Transformer48', 'name': 'mxCell_932', 'id': 'qValLhmmgod3405-91VG-1064', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_923', 'lv_bus': 'mxCell_926', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '243': {'typ': 'Transformer49', 'name': 'mxCell_948', 'id': 'qValLhmmgod3405-91VG-1080', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_939', 'lv_bus': 'mxCell_942', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '244': {'typ': 'Transformer50', 'name': 'mxCell_964', 'id': 'qValLhmmgod3405-91VG-1096', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_955', 'lv_bus': 'mxCell_958', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '245': {'typ': 'Transformer51', 'name': 'mxCell_990', 'id': 'qValLhmmgod3405-91VG-1122', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_969', 'lv_bus': 'mxCell_972', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '246': {'typ': 'Transformer52', 'name': 'mxCell_994', 'id': 'qValLhmmgod3405-91VG-1126', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_981', 'lv_bus': 'mxCell_984', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '247': {'typ': 'Transformer53', 'name': 'mxCell_1010', 'id': 'qValLhmmgod3405-91VG-1142', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1001', 'lv_bus': 'mxCell_1004', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '248': {'typ': 'Transformer54', 'name': 'mxCell_1026', 'id': 'qValLhmmgod3405-91VG-1158', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1017', 'lv_bus': 'mxCell_1020', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '249': {'typ': 'Transformer55', 'name': 'mxCell_1042', 'id': 'qValLhmmgod3405-91VG-1174', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1033', 'lv_bus': 'mxCell_1036', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '250': {'typ': 'Transformer56', 'name': 'mxCell_1068', 'id': 'qValLhmmgod3405-91VG-1200', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1047', 'lv_bus': 'mxCell_1050', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '251': {'typ': 'Transformer57', 'name': 'mxCell_1072', 'id': 'qValLhmmgod3405-91VG-1204', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1059', 'lv_bus': 'mxCell_1062', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '252': {'typ': 'Transformer58', 'name': 'mxCell_1088', 'id': 'qValLhmmgod3405-91VG-1220', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1079', 'lv_bus': 'mxCell_1082', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '253': {'typ': 'Transformer59', 'name': 'mxCell_1104', 'id': 'qValLhmmgod3405-91VG-1236', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1095', 'lv_bus': 'mxCell_1098', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '254': {'typ': 'Transformer60', 'name': 'mxCell_1120', 'id': 'qValLhmmgod3405-91VG-1252', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1111', 'lv_bus': 'mxCell_1114', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '255': {'typ': 'Transformer61', 'name': 'mxCell_1146', 'id': 'qValLhmmgod3405-91VG-1278', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1125', 'lv_bus': 'mxCell_1128', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '256': {'typ': 'Transformer62', 'name': 'mxCell_1150', 'id': 'qValLhmmgod3405-91VG-1282', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1137', 'lv_bus': 'mxCell_1140', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '257': {'typ': 'Transformer63', 'name': 'mxCell_1166', 'id': 'qValLhmmgod3405-91VG-1298', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1157', 'lv_bus': 'mxCell_1160', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '258': {'typ': 'Transformer64', 'name': 'mxCell_1182', 'id': 'qValLhmmgod3405-91VG-1314', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1173', 'lv_bus': 'mxCell_1176', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '259': {'typ': 'Transformer65', 'name': 'mxCell_1198', 'id': 'qValLhmmgod3405-91VG-1330', 'userFriendlyName': 'WTG transformer', 'hv_bus': 'mxCell_1189', 'lv_bus': 'mxCell_1192', 'sn_mva': '16.8', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'vkr_percent': '1', 'vk_percent': '8', 'pfe_kw': '76', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '6', 'tap_neutral': '0', 'tap_max': '6', 'tap_min': '-6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '260': {'typ': 'Transformer66', 'name': 'mxCell_1202', 'id': 'qValLhmmgod3405-91VG-1334', 'userFriendlyName': '300MVA 275/66kV', 'hv_bus': 'mxCell_697', 'lv_bus': 'mxCell_713', 'sn_mva': '300', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '261': {'typ': 'Transformer67', 'name': 'mxCell_1206', 'id': 'qValLhmmgod3405-91VG-1338', 'userFriendlyName': '300MVA 400/275kV', 'hv_bus': 'mxCell_148', 'lv_bus': 'mxCell_689', 'sn_mva': '300', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'vkr_percent': '1', 'vk_percent': '12', 'pfe_kw': '100', 'i0_percent': '1', 'vector_group': 'Dyn', 'vk0_percent': '0', 'vkr0_percent': '0', 'mag0_percent': '0', 'si0_hv_partial': '0', 'parallel': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_pos': '0', 'tap_neutral': '0', 'tap_max': '10', 'tap_min': '-10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_phase_shifter': 'false', 'tap_changer_type': 'Ratio', 'in_service': 'true', 'discrete_tap_control': 'false', 'control_side': 'lv', 'vm_lower_pu': '0.99', 'vm_upper_pu': '1.01'}, '262': {'typ': 'Shunt Reactor0', 'name': 'mxCell_170', 'id': 'aSc5nKb4JZ8KHuJaPj6X-35', 'userFriendlyName': 'Shunt Reactor', 'bus': 'mxCell_157', 'p_mw': '0.1', 'q_mvar': '300', 'vn_kv': '275', 'step': '1', 'max_step': '1', 'in_service': 'true'}, '263': {'typ': 'Shunt Reactor1', 'name': 'mxCell_680', 'id': 'qValLhmmgod3405-91VG-804', 'userFriendlyName': 'Shunt Reactor', 'bus': 'mxCell_165', 'p_mw': '1', 'q_mvar': '200', 'vn_kv': '275', 'step': '1', 'max_step': '1', 'in_service': 'true'}, '264': {'typ': 'Shunt Reactor2', 'name': 'mxCell_702', 'id': 'qValLhmmgod3405-91VG-833', 'userFriendlyName': 'Shunt Reactor', 'bus': 'mxCell_689', 'p_mw': '0.1', 'q_mvar': '300', 'vn_kv': '275', 'step': '1', 'max_step': '1', 'in_service': 'true'}, '265': {'typ': 'Shunt Reactor3', 'name': 'mxCell_1209', 'id': 'qValLhmmgod3405-91VG-1341', 'userFriendlyName': 'Shunt Reactor', 'bus': 'mxCell_697', 'p_mw': '1', 'q_mvar': '200', 'vn_kv': '275', 'step': '1', 'max_step': '1', 'in_service': 'true'}, '266': {'typ': 'Line0', 'name': 'mxCell_143', 'id': 'aSc5nKb4JZ8KHuJaPj6X-4', 'userFriendlyName': '400', 'busFrom': 'mxCell_145', 'busTo': 'mxCell_148', 'length_km': '10', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.06', 'x_ohm_per_km': '0.144', 'c_nf_per_km': '144', 'g_us_per_km': '0', 'max_i_ka': '0.8', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '267': {'typ': 'Line1', 'name': 'mxCell_155', 'id': 'aSc5nKb4JZ8KHuJaPj6X-14', 'userFriendlyName': 'Onshore 275kV - 1x1800 RM Al', 'busFrom': 'mxCell_157', 'busTo': 'mxCell_190', 'length_km': '10', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.02', 'x_ohm_per_km': '0.59', 'c_nf_per_km': '210', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '268': {'typ': 'Line2', 'name': 'mxCell_160', 'id': 'aSc5nKb4JZ8KHuJaPj6X-23', 'userFriendlyName': 'Offshore 275kV - 3x1600 RMS Cu', 'busFrom': 'mxCell_162', 'busTo': 'mxCell_165', 'length_km': '89', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.021', 'x_ohm_per_km': '0.116', 'c_nf_per_km': '210', 'g_us_per_km': '0', 'max_i_ka': '1.2', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '269': {'typ': 'Line3', 'name': 'mxCell_171', 'id': 'aSc5nKb4JZ8KHuJaPj6X-73', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Al', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_194', 'length_km': '2', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '333', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '270': {'typ': 'Line4', 'name': 'mxCell_173', 'id': 'qValLhmmgod3405-91VG-310', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_272', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '271': {'typ': 'Line5', 'name': 'mxCell_175', 'id': 'qValLhmmgod3405-91VG-788', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_438', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '272': {'typ': 'Line6', 'name': 'mxCell_177', 'id': 'qValLhmmgod3405-91VG-790', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Al', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_516', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '333', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '273': {'typ': 'Line7', 'name': 'mxCell_179', 'id': 'qValLhmmgod3405-91VG-792', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_594', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '274': {'typ': 'Line8', 'name': 'mxCell_188', 'id': 'aSc5nKb4JZ8KHuJaPj6X-55', 'userFriendlyName': 'Offshore 275kV - 3x2500 RMS Cu', 'busFrom': 'mxCell_190', 'busTo': 'mxCell_162', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.01', 'x_ohm_per_km': '0.1', 'c_nf_per_km': '256', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '275': {'typ': 'Line9', 'name': 'mxCell_204', 'id': 'aSc5nKb4JZ8KHuJaPj6X-111', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_194', 'busTo': 'mxCell_206', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '276': {'typ': 'Line10', 'name': 'mxCell_224', 'id': 'aSc5nKb4JZ8KHuJaPj6X-139', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_206', 'busTo': 'mxCell_226', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '277': {'typ': 'Line11', 'name': 'mxCell_240', 'id': 'aSc5nKb4JZ8KHuJaPj6X-155', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_226', 'busTo': 'mxCell_242', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '278': {'typ': 'Line12', 'name': 'mxCell_256', 'id': 'qValLhmmgod3405-91VG-4', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_242', 'busTo': 'mxCell_258', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '279': {'typ': 'Line13', 'name': 'mxCell_282', 'id': 'qValLhmmgod3405-91VG-228', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_272', 'busTo': 'mxCell_284', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '280': {'typ': 'Line14', 'name': 'mxCell_302', 'id': 'qValLhmmgod3405-91VG-248', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_284', 'busTo': 'mxCell_304', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '281': {'typ': 'Line15', 'name': 'mxCell_318', 'id': 'qValLhmmgod3405-91VG-264', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_304', 'busTo': 'mxCell_320', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '282': {'typ': 'Line16', 'name': 'mxCell_334', 'id': 'qValLhmmgod3405-91VG-280', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_320', 'busTo': 'mxCell_336', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '283': {'typ': 'Line17', 'name': 'mxCell_354', 'id': 'qValLhmmgod3405-91VG-312', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_360', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '284': {'typ': 'Line18', 'name': 'mxCell_370', 'id': 'qValLhmmgod3405-91VG-328', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_360', 'busTo': 'mxCell_372', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '285': {'typ': 'Line19', 'name': 'mxCell_390', 'id': 'qValLhmmgod3405-91VG-348', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_372', 'busTo': 'mxCell_392', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '286': {'typ': 'Line20', 'name': 'mxCell_406', 'id': 'qValLhmmgod3405-91VG-364', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_392', 'busTo': 'mxCell_408', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '287': {'typ': 'Line21', 'name': 'mxCell_422', 'id': 'qValLhmmgod3405-91VG-380', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_408', 'busTo': 'mxCell_424', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '288': {'typ': 'Line22', 'name': 'mxCell_448', 'id': 'qValLhmmgod3405-91VG-568', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_438', 'busTo': 'mxCell_450', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '289': {'typ': 'Line23', 'name': 'mxCell_468', 'id': 'qValLhmmgod3405-91VG-588', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_450', 'busTo': 'mxCell_470', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '290': {'typ': 'Line24', 'name': 'mxCell_484', 'id': 'qValLhmmgod3405-91VG-604', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_470', 'busTo': 'mxCell_486', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '291': {'typ': 'Line25', 'name': 'mxCell_500', 'id': 'qValLhmmgod3405-91VG-620', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_486', 'busTo': 'mxCell_502', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '292': {'typ': 'Line26', 'name': 'mxCell_526', 'id': 'qValLhmmgod3405-91VG-646', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_516', 'busTo': 'mxCell_528', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '293': {'typ': 'Line27', 'name': 'mxCell_546', 'id': 'qValLhmmgod3405-91VG-666', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_528', 'busTo': 'mxCell_548', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '294': {'typ': 'Line28', 'name': 'mxCell_562', 'id': 'qValLhmmgod3405-91VG-682', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_548', 'busTo': 'mxCell_564', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '295': {'typ': 'Line29', 'name': 'mxCell_578', 'id': 'qValLhmmgod3405-91VG-698', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_564', 'busTo': 'mxCell_580', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '296': {'typ': 'Line30', 'name': 'mxCell_604', 'id': 'qValLhmmgod3405-91VG-724', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_594', 'busTo': 'mxCell_606', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '297': {'typ': 'Line31', 'name': 'mxCell_624', 'id': 'qValLhmmgod3405-91VG-744', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_606', 'busTo': 'mxCell_626', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '298': {'typ': 'Line32', 'name': 'mxCell_640', 'id': 'qValLhmmgod3405-91VG-760', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_626', 'busTo': 'mxCell_642', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '299': {'typ': 'Line33', 'name': 'mxCell_656', 'id': 'qValLhmmgod3405-91VG-776', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_642', 'busTo': 'mxCell_658', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '300': {'typ': 'Line34', 'name': 'mxCell_676', 'id': 'qValLhmmgod3405-91VG-802', 'userFriendlyName': '400', 'busFrom': 'mxCell_145', 'busTo': 'mxCell_148', 'length_km': '10', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.06', 'x_ohm_per_km': '0.144', 'c_nf_per_km': '144', 'g_us_per_km': '0', 'max_i_ka': '0.8', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '301': {'typ': 'Line35', 'name': 'mxCell_687', 'id': 'qValLhmmgod3405-91VG-818', 'userFriendlyName': 'Onshore 275kV - 1x1800 RM Al', 'busFrom': 'mxCell_689', 'busTo': 'mxCell_721', 'length_km': '10', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.02', 'x_ohm_per_km': '0.59', 'c_nf_per_km': '210', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '302': {'typ': 'Line36', 'name': 'mxCell_692', 'id': 'qValLhmmgod3405-91VG-823', 'userFriendlyName': 'Offshore 275kV - 3x1600 RMS Cu', 'busFrom': 'mxCell_694', 'busTo': 'mxCell_697', 'length_km': '89', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.021', 'x_ohm_per_km': '0.116', 'c_nf_per_km': '210', 'g_us_per_km': '0', 'max_i_ka': '1.2', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '303': {'typ': 'Line37', 'name': 'mxCell_703', 'id': 'qValLhmmgod3405-91VG-834', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Al', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_725', 'length_km': '2', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '333', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '304': {'typ': 'Line38', 'name': 'mxCell_705', 'id': 'qValLhmmgod3405-91VG-836', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_803', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '305': {'typ': 'Line39', 'name': 'mxCell_707', 'id': 'qValLhmmgod3405-91VG-838', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_969', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '306': {'typ': 'Line40', 'name': 'mxCell_709', 'id': 'qValLhmmgod3405-91VG-840', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Al', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_1047', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '333', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '307': {'typ': 'Line41', 'name': 'mxCell_711', 'id': 'qValLhmmgod3405-91VG-842', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_1125', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '308': {'typ': 'Line42', 'name': 'mxCell_719', 'id': 'qValLhmmgod3405-91VG-851', 'userFriendlyName': 'Offshore 275kV - 3x2500 RMS Cu', 'busFrom': 'mxCell_721', 'busTo': 'mxCell_694', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.01', 'x_ohm_per_km': '0.1', 'c_nf_per_km': '256', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '309': {'typ': 'Line43', 'name': 'mxCell_735', 'id': 'qValLhmmgod3405-91VG-867', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_725', 'busTo': 'mxCell_737', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '310': {'typ': 'Line44', 'name': 'mxCell_755', 'id': 'qValLhmmgod3405-91VG-887', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_737', 'busTo': 'mxCell_757', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '311': {'typ': 'Line45', 'name': 'mxCell_771', 'id': 'qValLhmmgod3405-91VG-903', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_757', 'busTo': 'mxCell_773', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '312': {'typ': 'Line46', 'name': 'mxCell_787', 'id': 'qValLhmmgod3405-91VG-919', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_773', 'busTo': 'mxCell_789', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '313': {'typ': 'Line47', 'name': 'mxCell_813', 'id': 'qValLhmmgod3405-91VG-945', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_803', 'busTo': 'mxCell_815', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '314': {'typ': 'Line48', 'name': 'mxCell_833', 'id': 'qValLhmmgod3405-91VG-965', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_815', 'busTo': 'mxCell_835', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '315': {'typ': 'Line49', 'name': 'mxCell_849', 'id': 'qValLhmmgod3405-91VG-981', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_835', 'busTo': 'mxCell_851', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '316': {'typ': 'Line50', 'name': 'mxCell_865', 'id': 'qValLhmmgod3405-91VG-997', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_851', 'busTo': 'mxCell_867', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '317': {'typ': 'Line51', 'name': 'mxCell_885', 'id': 'qValLhmmgod3405-91VG-1017', 'userFriendlyName': 'Offshore 66kV - 3x800 RM Cu', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_891', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'c_nf_per_km': '343', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '318': {'typ': 'Line52', 'name': 'mxCell_901', 'id': 'qValLhmmgod3405-91VG-1033', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_891', 'busTo': 'mxCell_903', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '319': {'typ': 'Line53', 'name': 'mxCell_921', 'id': 'qValLhmmgod3405-91VG-1053', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_903', 'busTo': 'mxCell_923', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '320': {'typ': 'Line54', 'name': 'mxCell_937', 'id': 'qValLhmmgod3405-91VG-1069', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_923', 'busTo': 'mxCell_939', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '321': {'typ': 'Line55', 'name': 'mxCell_953', 'id': 'qValLhmmgod3405-91VG-1085', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_939', 'busTo': 'mxCell_955', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '322': {'typ': 'Line56', 'name': 'mxCell_979', 'id': 'qValLhmmgod3405-91VG-1111', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_969', 'busTo': 'mxCell_981', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '323': {'typ': 'Line57', 'name': 'mxCell_999', 'id': 'qValLhmmgod3405-91VG-1131', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_981', 'busTo': 'mxCell_1001', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '324': {'typ': 'Line58', 'name': 'mxCell_1015', 'id': 'qValLhmmgod3405-91VG-1147', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1001', 'busTo': 'mxCell_1017', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '325': {'typ': 'Line59', 'name': 'mxCell_1031', 'id': 'qValLhmmgod3405-91VG-1163', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1017', 'busTo': 'mxCell_1033', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '326': {'typ': 'Line60', 'name': 'mxCell_1057', 'id': 'qValLhmmgod3405-91VG-1189', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_1047', 'busTo': 'mxCell_1059', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '327': {'typ': 'Line61', 'name': 'mxCell_1077', 'id': 'qValLhmmgod3405-91VG-1209', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1059', 'busTo': 'mxCell_1079', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '328': {'typ': 'Line62', 'name': 'mxCell_1093', 'id': 'qValLhmmgod3405-91VG-1225', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1079', 'busTo': 'mxCell_1095', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '329': {'typ': 'Line63', 'name': 'mxCell_1109', 'id': 'qValLhmmgod3405-91VG-1241', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1095', 'busTo': 'mxCell_1111', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '330': {'typ': 'Line64', 'name': 'mxCell_1135', 'id': 'qValLhmmgod3405-91VG-1267', 'userFriendlyName': 'Offshore 275kV - 3x800 RMS Cu', 'busFrom': 'mxCell_1125', 'busTo': 'mxCell_1137', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'c_nf_per_km': '159', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '331': {'typ': 'Line65', 'name': 'mxCell_1155', 'id': 'qValLhmmgod3405-91VG-1287', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1137', 'busTo': 'mxCell_1157', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '332': {'typ': 'Line66', 'name': 'mxCell_1171', 'id': 'qValLhmmgod3405-91VG-1303', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1157', 'busTo': 'mxCell_1173', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '333': {'typ': 'Line67', 'name': 'mxCell_1187', 'id': 'qValLhmmgod3405-91VG-1319', 'userFriendlyName': 'Offshore 66kV - 3x300 RM Al', 'busFrom': 'mxCell_1173', 'busTo': 'mxCell_1189', 'length_km': '1', 'parallel': '1', 'df': '1', 'in_service': 'true', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'c_nf_per_km': '228', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}
}

in_data_opendss = {
    '0': {'typ': 'PowerFlowOpenDss Parameters', 'frequency': '50', 'mode': 'Snapshot', 'algorithm': 'Normal', 'loadmodel': 'Powerflow', 'maxIterations': '100', 'tolerance': '0.0001', 'controlmode': 'Static', 'exportCommands': False, 'exportOpenDSSResults': False, 'user_email': 'maciej@gmail.com'}, '1': {'typ': 'External Grid', 'name': 'mxCell_141', 'id': 'mxCell#141', 'bus': 'mxCell_145', 'vm_pu': '1', 'va_degree': '0', 's_sc_max_mva': '100000000', 's_sc_min_mva': '0', 'rx_max': '0', 'rx_min': '0', 'r0x0_max': '0', 'x0x_max': '0', 'in_service': True}, '2': {'typ': 'Line', 'name': 'mxCell_143', 'id': 'mxCell#143', 'busFrom': 'mxCell_145', 'busTo': 'mxCell_148', 'r_ohm_per_km': '0.06', 'x_ohm_per_km': '0.144', 'length_km': '10', 'c_nf_per_km': '144', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.8', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '3': {'typ': 'Bus0', 'name': 'mxCell_145', 'id': 'mxCell#145', 'vn_kv': '400', 'userFriendlyName': 'Bus'}, '4': {'typ': 'Bus1', 'name': 'mxCell_148', 'id': 'mxCell#148', 'vn_kv': '400', 'userFriendlyName': 'Bus'}, '5': {'typ': 'Static Generator', 'name': 'mxCell_153', 'id': 'mxCell#153', 'bus': 'mxCell_197', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '6': {'typ': 'Line', 'name': 'mxCell_155', 'id': 'mxCell#155', 'busFrom': 'mxCell_157', 'busTo': 'mxCell_190', 'r_ohm_per_km': '0.02', 'x_ohm_per_km': '0.59', 'length_km': '10', 'c_nf_per_km': '210', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '7': {'typ': 'Bus2', 'name': 'mxCell_157', 'id': 'mxCell#157', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '8': {'typ': 'Line', 'name': 'mxCell_160', 'id': 'mxCell#160', 'busFrom': 'mxCell_162', 'busTo': 'mxCell_165', 'r_ohm_per_km': '0.021', 'x_ohm_per_km': '0.116', 'length_km': '89', 'c_nf_per_km': '210', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '1.2', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '9': {'typ': 'Bus3', 'name': 'mxCell_162', 'id': 'mxCell#162', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '10': {'typ': 'Bus4', 'name': 'mxCell_165', 'id': 'mxCell#165', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '11': {'typ': 'Shunt Reactor', 'name': 'mxCell_170', 'id': 'mxCell#170', 'bus': 'mxCell_157', 'p_mw': '0.1', 'q_mvar': '300', 'sn_mva': 1, 'scaling': 1, 'type': 'const_q', 'step': '1', 'step_degree': 0, 'step_min': 0, 'step_max': 1, 'step_pos': 1, 'step_phase_shifter': False, 'in_service': True}, '12': {'typ': 'Line', 'name': 'mxCell_171', 'id': 'mxCell#171', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_194', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'length_km': '2', 'c_nf_per_km': '333', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '13': {'typ': 'Line', 'name': 'mxCell_173', 'id': 'mxCell#173', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_272', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '14': {'typ': 'Line', 'name': 'mxCell_175', 'id': 'mxCell#175', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_438', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '15': {'typ': 'Line', 'name': 'mxCell_177', 'id': 'mxCell#177', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_516', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '333', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '16': {'typ': 'Line', 'name': 'mxCell_179', 'id': 'mxCell#179', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_594', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '17': {'typ': 'Bus5', 'name': 'mxCell_181', 'id': 'mxCell#181', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '18': {'typ': 'Line', 'name': 'mxCell_188', 'id': 'mxCell#188', 'busFrom': 'mxCell_190', 'busTo': 'mxCell_162', 'r_ohm_per_km': '0.01', 'x_ohm_per_km': '0.1', 'length_km': '1', 'c_nf_per_km': '256', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '19': {'typ': 'Bus6', 'name': 'mxCell_190', 'id': 'mxCell#190', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '20': {'typ': 'Bus7', 'name': 'mxCell_194', 'id': 'mxCell#194', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '21': {'typ': 'Bus8', 'name': 'mxCell_197', 'id': 'mxCell#197', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '22': {'typ': 'Static Generator', 'name': 'mxCell_202', 'id': 'mxCell#202', 'bus': 'mxCell_209', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '23': {'typ': 'Line', 'name': 'mxCell_204', 'id': 'mxCell#204', 'busFrom': 'mxCell_194', 'busTo': 'mxCell_206', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '24': {'typ': 'Bus9', 'name': 'mxCell_206', 'id': 'mxCell#206', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '25': {'typ': 'Bus10', 'name': 'mxCell_209', 'id': 'mxCell#209', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '26': {'typ': 'Transformer', 'name': 'mxCell_215', 'id': 'mxCell#215', 'busFrom': 'mxCell_194', 'busTo': 'mxCell_197', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '27': {'typ': 'Transformer', 'name': 'mxCell_219', 'id': 'mxCell#219', 'busFrom': 'mxCell_206', 'busTo': 'mxCell_209', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '28': {'typ': 'Static Generator', 'name': 'mxCell_222', 'id': 'mxCell#222', 'bus': 'mxCell_229', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '29': {'typ': 'Line', 'name': 'mxCell_224', 'id': 'mxCell#224', 'busFrom': 'mxCell_206', 'busTo': 'mxCell_226', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '30': {'typ': 'Bus11', 'name': 'mxCell_226', 'id': 'mxCell#226', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '31': {'typ': 'Bus12', 'name': 'mxCell_229', 'id': 'mxCell#229', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '32': {'typ': 'Transformer', 'name': 'mxCell_235', 'id': 'mxCell#235', 'busFrom': 'mxCell_226', 'busTo': 'mxCell_229', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '33': {'typ': 'Static Generator', 'name': 'mxCell_238', 'id': 'mxCell#238', 'bus': 'mxCell_245', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '34': {'typ': 'Line', 'name': 'mxCell_240', 'id': 'mxCell#240', 'busFrom': 'mxCell_226', 'busTo': 'mxCell_242', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '35': {'typ': 'Bus13', 'name': 'mxCell_242', 'id': 'mxCell#242', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '36': {'typ': 'Bus14', 'name': 'mxCell_245', 'id': 'mxCell#245', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '37': {'typ': 'Transformer', 'name': 'mxCell_251', 'id': 'mxCell#251', 'busFrom': 'mxCell_242', 'busTo': 'mxCell_245', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '38': {'typ': 'Static Generator', 'name': 'mxCell_254', 'id': 'mxCell#254', 'bus': 'mxCell_261', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '39': {'typ': 'Line', 'name': 'mxCell_256', 'id': 'mxCell#256', 'busFrom': 'mxCell_242', 'busTo': 'mxCell_258', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '40': {'typ': 'Bus15', 'name': 'mxCell_258', 'id': 'mxCell#258', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '41': {'typ': 'Bus16', 'name': 'mxCell_261', 'id': 'mxCell#261', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '42': {'typ': 'Transformer', 'name': 'mxCell_267', 'id': 'mxCell#267', 'busFrom': 'mxCell_258', 'busTo': 'mxCell_261', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '43': {'typ': 'Static Generator', 'name': 'mxCell_270', 'id': 'mxCell#270', 'bus': 'mxCell_275', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '44': {'typ': 'Bus17', 'name': 'mxCell_272', 'id': 'mxCell#272', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '45': {'typ': 'Bus18', 'name': 'mxCell_275', 'id': 'mxCell#275', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '46': {'typ': 'Static Generator', 'name': 'mxCell_280', 'id': 'mxCell#280', 'bus': 'mxCell_287', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '47': {'typ': 'Line', 'name': 'mxCell_282', 'id': 'mxCell#282', 'busFrom': 'mxCell_272', 'busTo': 'mxCell_284', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '48': {'typ': 'Bus19', 'name': 'mxCell_284', 'id': 'mxCell#284', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '49': {'typ': 'Bus20', 'name': 'mxCell_287', 'id': 'mxCell#287', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '50': {'typ': 'Transformer', 'name': 'mxCell_293', 'id': 'mxCell#293', 'busFrom': 'mxCell_272', 'busTo': 'mxCell_275', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '51': {'typ': 'Transformer', 'name': 'mxCell_297', 'id': 'mxCell#297', 'busFrom': 'mxCell_284', 'busTo': 'mxCell_287', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '52': {'typ': 'Static Generator', 'name': 'mxCell_300', 'id': 'mxCell#300', 'bus': 'mxCell_307', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '53': {'typ': 'Line', 'name': 'mxCell_302', 'id': 'mxCell#302', 'busFrom': 'mxCell_284', 'busTo': 'mxCell_304', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '54': {'typ': 'Bus21', 'name': 'mxCell_304', 'id': 'mxCell#304', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '55': {'typ': 'Bus22', 'name': 'mxCell_307', 'id': 'mxCell#307', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '56': {'typ': 'Transformer', 'name': 'mxCell_313', 'id': 'mxCell#313', 'busFrom': 'mxCell_304', 'busTo': 'mxCell_307', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '57': {'typ': 'Static Generator', 'name': 'mxCell_316', 'id': 'mxCell#316', 'bus': 'mxCell_323', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '58': {'typ': 'Line', 'name': 'mxCell_318', 'id': 'mxCell#318', 'busFrom': 'mxCell_304', 'busTo': 'mxCell_320', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '59': {'typ': 'Bus23', 'name': 'mxCell_320', 'id': 'mxCell#320', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '60': {'typ': 'Bus24', 'name': 'mxCell_323', 'id': 'mxCell#323', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '61': {'typ': 'Transformer', 'name': 'mxCell_329', 'id': 'mxCell#329', 'busFrom': 'mxCell_320', 'busTo': 'mxCell_323', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '62': {'typ': 'Static Generator', 'name': 'mxCell_332', 'id': 'mxCell#332', 'bus': 'mxCell_339', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '63': {'typ': 'Line', 'name': 'mxCell_334', 'id': 'mxCell#334', 'busFrom': 'mxCell_320', 'busTo': 'mxCell_336', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '64': {'typ': 'Bus25', 'name': 'mxCell_336', 'id': 'mxCell#336', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '65': {'typ': 'Bus26', 'name': 'mxCell_339', 'id': 'mxCell#339', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '66': {'typ': 'Transformer', 'name': 'mxCell_345', 'id': 'mxCell#345', 'busFrom': 'mxCell_336', 'busTo': 'mxCell_339', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '67': {'typ': 'Transformer', 'name': 'mxCell_349', 'id': 'mxCell#349', 'busFrom': 'mxCell_148', 'busTo': 'mxCell_157', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '68': {'typ': 'Transformer', 'name': 'mxCell_353', 'id': 'mxCell#353', 'busFrom': 'mxCell_165', 'busTo': 'mxCell_181', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '69': {'typ': 'Line', 'name': 'mxCell_354', 'id': 'mxCell#354', 'busFrom': 'mxCell_181', 'busTo': 'mxCell_360', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '70': {'typ': 'Static Generator', 'name': 'mxCell_358', 'id': 'mxCell#358', 'bus': 'mxCell_363', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '71': {'typ': 'Bus27', 'name': 'mxCell_360', 'id': 'mxCell#360', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '72': {'typ': 'Bus28', 'name': 'mxCell_363', 'id': 'mxCell#363', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '73': {'typ': 'Static Generator', 'name': 'mxCell_368', 'id': 'mxCell#368', 'bus': 'mxCell_375', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '74': {'typ': 'Line', 'name': 'mxCell_370', 'id': 'mxCell#370', 'busFrom': 'mxCell_360', 'busTo': 'mxCell_372', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '75': {'typ': 'Bus29', 'name': 'mxCell_372', 'id': 'mxCell#372', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '76': {'typ': 'Bus30', 'name': 'mxCell_375', 'id': 'mxCell#375', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '77': {'typ': 'Transformer', 'name': 'mxCell_381', 'id': 'mxCell#381', 'busFrom': 'mxCell_360', 'busTo': 'mxCell_363', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '78': {'typ': 'Transformer', 'name': 'mxCell_385', 'id': 'mxCell#385', 'busFrom': 'mxCell_372', 'busTo': 'mxCell_375', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '79': {'typ': 'Static Generator', 'name': 'mxCell_388', 'id': 'mxCell#388', 'bus': 'mxCell_395', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '80': {'typ': 'Line', 'name': 'mxCell_390', 'id': 'mxCell#390', 'busFrom': 'mxCell_372', 'busTo': 'mxCell_392', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '81': {'typ': 'Bus31', 'name': 'mxCell_392', 'id': 'mxCell#392', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '82': {'typ': 'Bus32', 'name': 'mxCell_395', 'id': 'mxCell#395', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '83': {'typ': 'Transformer', 'name': 'mxCell_401', 'id': 'mxCell#401', 'busFrom': 'mxCell_392', 'busTo': 'mxCell_395', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '84': {'typ': 'Static Generator', 'name': 'mxCell_404', 'id': 'mxCell#404', 'bus': 'mxCell_411', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '85': {'typ': 'Line', 'name': 'mxCell_406', 'id': 'mxCell#406', 'busFrom': 'mxCell_392', 'busTo': 'mxCell_408', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '86': {'typ': 'Bus33', 'name': 'mxCell_408', 'id': 'mxCell#408', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '87': {'typ': 'Bus34', 'name': 'mxCell_411', 'id': 'mxCell#411', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '88': {'typ': 'Transformer', 'name': 'mxCell_417', 'id': 'mxCell#417', 'busFrom': 'mxCell_408', 'busTo': 'mxCell_411', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '89': {'typ': 'Static Generator', 'name': 'mxCell_420', 'id': 'mxCell#420', 'bus': 'mxCell_427', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '90': {'typ': 'Line', 'name': 'mxCell_422', 'id': 'mxCell#422', 'busFrom': 'mxCell_408', 'busTo': 'mxCell_424', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '91': {'typ': 'Bus35', 'name': 'mxCell_424', 'id': 'mxCell#424', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '92': {'typ': 'Bus36', 'name': 'mxCell_427', 'id': 'mxCell#427', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '93': {'typ': 'Transformer', 'name': 'mxCell_433', 'id': 'mxCell#433', 'busFrom': 'mxCell_424', 'busTo': 'mxCell_427', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '94': {'typ': 'Static Generator', 'name': 'mxCell_436', 'id': 'mxCell#436', 'bus': 'mxCell_441', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '95': {'typ': 'Bus37', 'name': 'mxCell_438', 'id': 'mxCell#438', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '96': {'typ': 'Bus38', 'name': 'mxCell_441', 'id': 'mxCell#441', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '97': {'typ': 'Static Generator', 'name': 'mxCell_446', 'id': 'mxCell#446', 'bus': 'mxCell_453', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '98': {'typ': 'Line', 'name': 'mxCell_448', 'id': 'mxCell#448', 'busFrom': 'mxCell_438', 'busTo': 'mxCell_450', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '99': {'typ': 'Bus39', 'name': 'mxCell_450', 'id': 'mxCell#450', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '100': {'typ': 'Bus40', 'name': 'mxCell_453', 'id': 'mxCell#453', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '101': {'typ': 'Transformer', 'name': 'mxCell_459', 'id': 'mxCell#459', 'busFrom': 'mxCell_438', 'busTo': 'mxCell_441', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '102': {'typ': 'Transformer', 'name': 'mxCell_463', 'id': 'mxCell#463', 'busFrom': 'mxCell_450', 'busTo': 'mxCell_453', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '103': {'typ': 'Static Generator', 'name': 'mxCell_466', 'id': 'mxCell#466', 'bus': 'mxCell_473', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '104': {'typ': 'Line', 'name': 'mxCell_468', 'id': 'mxCell#468', 'busFrom': 'mxCell_450', 'busTo': 'mxCell_470', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '105': {'typ': 'Bus41', 'name': 'mxCell_470', 'id': 'mxCell#470', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '106': {'typ': 'Bus42', 'name': 'mxCell_473', 'id': 'mxCell#473', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '107': {'typ': 'Transformer', 'name': 'mxCell_479', 'id': 'mxCell#479', 'busFrom': 'mxCell_470', 'busTo': 'mxCell_473', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '108': {'typ': 'Static Generator', 'name': 'mxCell_482', 'id': 'mxCell#482', 'bus': 'mxCell_489', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '109': {'typ': 'Line', 'name': 'mxCell_484', 'id': 'mxCell#484', 'busFrom': 'mxCell_470', 'busTo': 'mxCell_486', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '110': {'typ': 'Bus43', 'name': 'mxCell_486', 'id': 'mxCell#486', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '111': {'typ': 'Bus44', 'name': 'mxCell_489', 'id': 'mxCell#489', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '112': {'typ': 'Transformer', 'name': 'mxCell_495', 'id': 'mxCell#495', 'busFrom': 'mxCell_486', 'busTo': 'mxCell_489', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '113': {'typ': 'Static Generator', 'name': 'mxCell_498', 'id': 'mxCell#498', 'bus': 'mxCell_505', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '114': {'typ': 'Line', 'name': 'mxCell_500', 'id': 'mxCell#500', 'busFrom': 'mxCell_486', 'busTo': 'mxCell_502', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '115': {'typ': 'Bus45', 'name': 'mxCell_502', 'id': 'mxCell#502', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '116': {'typ': 'Bus46', 'name': 'mxCell_505', 'id': 'mxCell#505', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '117': {'typ': 'Transformer', 'name': 'mxCell_511', 'id': 'mxCell#511', 'busFrom': 'mxCell_502', 'busTo': 'mxCell_505', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '118': {'typ': 'Static Generator', 'name': 'mxCell_514', 'id': 'mxCell#514', 'bus': 'mxCell_519', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '119': {'typ': 'Bus47', 'name': 'mxCell_516', 'id': 'mxCell#516', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '120': {'typ': 'Bus48', 'name': 'mxCell_519', 'id': 'mxCell#519', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '121': {'typ': 'Static Generator', 'name': 'mxCell_524', 'id': 'mxCell#524', 'bus': 'mxCell_531', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '122': {'typ': 'Line', 'name': 'mxCell_526', 'id': 'mxCell#526', 'busFrom': 'mxCell_516', 'busTo': 'mxCell_528', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '123': {'typ': 'Bus49', 'name': 'mxCell_528', 'id': 'mxCell#528', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '124': {'typ': 'Bus50', 'name': 'mxCell_531', 'id': 'mxCell#531', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '125': {'typ': 'Transformer', 'name': 'mxCell_537', 'id': 'mxCell#537', 'busFrom': 'mxCell_516', 'busTo': 'mxCell_519', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '126': {'typ': 'Transformer', 'name': 'mxCell_541', 'id': 'mxCell#541', 'busFrom': 'mxCell_528', 'busTo': 'mxCell_531', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '127': {'typ': 'Static Generator', 'name': 'mxCell_544', 'id': 'mxCell#544', 'bus': 'mxCell_551', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '128': {'typ': 'Line', 'name': 'mxCell_546', 'id': 'mxCell#546', 'busFrom': 'mxCell_528', 'busTo': 'mxCell_548', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '129': {'typ': 'Bus51', 'name': 'mxCell_548', 'id': 'mxCell#548', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '130': {'typ': 'Bus52', 'name': 'mxCell_551', 'id': 'mxCell#551', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '131': {'typ': 'Transformer', 'name': 'mxCell_557', 'id': 'mxCell#557', 'busFrom': 'mxCell_548', 'busTo': 'mxCell_551', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '132': {'typ': 'Static Generator', 'name': 'mxCell_560', 'id': 'mxCell#560', 'bus': 'mxCell_567', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '133': {'typ': 'Line', 'name': 'mxCell_562', 'id': 'mxCell#562', 'busFrom': 'mxCell_548', 'busTo': 'mxCell_564', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '134': {'typ': 'Bus53', 'name': 'mxCell_564', 'id': 'mxCell#564', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '135': {'typ': 'Bus54', 'name': 'mxCell_567', 'id': 'mxCell#567', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '136': {'typ': 'Transformer', 'name': 'mxCell_573', 'id': 'mxCell#573', 'busFrom': 'mxCell_564', 'busTo': 'mxCell_567', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '137': {'typ': 'Static Generator', 'name': 'mxCell_576', 'id': 'mxCell#576', 'bus': 'mxCell_583', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '138': {'typ': 'Line', 'name': 'mxCell_578', 'id': 'mxCell#578', 'busFrom': 'mxCell_564', 'busTo': 'mxCell_580', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '139': {'typ': 'Bus55', 'name': 'mxCell_580', 'id': 'mxCell#580', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '140': {'typ': 'Bus56', 'name': 'mxCell_583', 'id': 'mxCell#583', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '141': {'typ': 'Transformer', 'name': 'mxCell_589', 'id': 'mxCell#589', 'busFrom': 'mxCell_580', 'busTo': 'mxCell_583', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '142': {'typ': 'Static Generator', 'name': 'mxCell_592', 'id': 'mxCell#592', 'bus': 'mxCell_597', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '143': {'typ': 'Bus57', 'name': 'mxCell_594', 'id': 'mxCell#594', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '144': {'typ': 'Bus58', 'name': 'mxCell_597', 'id': 'mxCell#597', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '145': {'typ': 'Static Generator', 'name': 'mxCell_602', 'id': 'mxCell#602', 'bus': 'mxCell_609', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '146': {'typ': 'Line', 'name': 'mxCell_604', 'id': 'mxCell#604', 'busFrom': 'mxCell_594', 'busTo': 'mxCell_606', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '147': {'typ': 'Bus59', 'name': 'mxCell_606', 'id': 'mxCell#606', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '148': {'typ': 'Bus60', 'name': 'mxCell_609', 'id': 'mxCell#609', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '149': {'typ': 'Transformer', 'name': 'mxCell_615', 'id': 'mxCell#615', 'busFrom': 'mxCell_594', 'busTo': 'mxCell_597', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '150': {'typ': 'Transformer', 'name': 'mxCell_619', 'id': 'mxCell#619', 'busFrom': 'mxCell_606', 'busTo': 'mxCell_609', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '151': {'typ': 'Static Generator', 'name': 'mxCell_622', 'id': 'mxCell#622', 'bus': 'mxCell_629', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '152': {'typ': 'Line', 'name': 'mxCell_624', 'id': 'mxCell#624', 'busFrom': 'mxCell_606', 'busTo': 'mxCell_626', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '153': {'typ': 'Bus61', 'name': 'mxCell_626', 'id': 'mxCell#626', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '154': {'typ': 'Bus62', 'name': 'mxCell_629', 'id': 'mxCell#629', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '155': {'typ': 'Transformer', 'name': 'mxCell_635', 'id': 'mxCell#635', 'busFrom': 'mxCell_626', 'busTo': 'mxCell_629', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '156': {'typ': 'Static Generator', 'name': 'mxCell_638', 'id': 'mxCell#638', 'bus': 'mxCell_645', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '157': {'typ': 'Line', 'name': 'mxCell_640', 'id': 'mxCell#640', 'busFrom': 'mxCell_626', 'busTo': 'mxCell_642', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '158': {'typ': 'Bus63', 'name': 'mxCell_642', 'id': 'mxCell#642', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '159': {'typ': 'Bus64', 'name': 'mxCell_645', 'id': 'mxCell#645', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '160': {'typ': 'Transformer', 'name': 'mxCell_651', 'id': 'mxCell#651', 'busFrom': 'mxCell_642', 'busTo': 'mxCell_645', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '161': {'typ': 'Static Generator', 'name': 'mxCell_654', 'id': 'mxCell#654', 'bus': 'mxCell_661', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '162': {'typ': 'Line', 'name': 'mxCell_656', 'id': 'mxCell#656', 'busFrom': 'mxCell_642', 'busTo': 'mxCell_658', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '163': {'typ': 'Bus65', 'name': 'mxCell_658', 'id': 'mxCell#658', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '164': {'typ': 'Bus66', 'name': 'mxCell_661', 'id': 'mxCell#661', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '165': {'typ': 'Transformer', 'name': 'mxCell_667', 'id': 'mxCell#667', 'busFrom': 'mxCell_658', 'busTo': 'mxCell_661', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '166': {'typ': 'Transformer', 'name': 'mxCell_671', 'id': 'mxCell#671', 'busFrom': 'mxCell_165', 'busTo': 'mxCell_181', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '167': {'typ': 'Transformer', 'name': 'mxCell_675', 'id': 'mxCell#675', 'busFrom': 'mxCell_148', 'busTo': 'mxCell_157', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '168': {'typ': 'Line', 'name': 'mxCell_676', 'id': 'mxCell#676', 'busFrom': 'mxCell_145', 'busTo': 'mxCell_148', 'r_ohm_per_km': '0.06', 'x_ohm_per_km': '0.144', 'length_km': '10', 'c_nf_per_km': '144', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.8', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '169': {'typ': 'Shunt Reactor', 'name': 'mxCell_680', 'id': 'mxCell#680', 'bus': 'mxCell_165', 'p_mw': '1', 'q_mvar': '200', 'sn_mva': 1, 'scaling': 1, 'type': 'const_q', 'step': '1', 'step_degree': 0, 'step_min': 0, 'step_max': 1, 'step_pos': 1, 'step_phase_shifter': False, 'in_service': True}, '170': {'typ': 'Static Generator', 'name': 'mxCell_685', 'id': 'mxCell#685', 'bus': 'mxCell_728', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '171': {'typ': 'Line', 'name': 'mxCell_687', 'id': 'mxCell#687', 'busFrom': 'mxCell_689', 'busTo': 'mxCell_721', 'r_ohm_per_km': '0.02', 'x_ohm_per_km': '0.59', 'length_km': '10', 'c_nf_per_km': '210', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '172': {'typ': 'Bus67', 'name': 'mxCell_689', 'id': 'mxCell#689', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '173': {'typ': 'Line', 'name': 'mxCell_692', 'id': 'mxCell#692', 'busFrom': 'mxCell_694', 'busTo': 'mxCell_697', 'r_ohm_per_km': '0.021', 'x_ohm_per_km': '0.116', 'length_km': '89', 'c_nf_per_km': '210', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '1.2', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '174': {'typ': 'Bus68', 'name': 'mxCell_694', 'id': 'mxCell#694', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '175': {'typ': 'Bus69', 'name': 'mxCell_697', 'id': 'mxCell#697', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '176': {'typ': 'Shunt Reactor', 'name': 'mxCell_702', 'id': 'mxCell#702', 'bus': 'mxCell_689', 'p_mw': '0.1', 'q_mvar': '300', 'sn_mva': 1, 'scaling': 1, 'type': 'const_q', 'step': '1', 'step_degree': 0, 'step_min': 0, 'step_max': 1, 'step_pos': 1, 'step_phase_shifter': False, 'in_service': True}, '177': {'typ': 'Line', 'name': 'mxCell_703', 'id': 'mxCell#703', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_725', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'length_km': '2', 'c_nf_per_km': '333', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '178': {'typ': 'Line', 'name': 'mxCell_705', 'id': 'mxCell#705', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_803', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '179': {'typ': 'Line', 'name': 'mxCell_707', 'id': 'mxCell#707', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_969', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '180': {'typ': 'Line', 'name': 'mxCell_709', 'id': 'mxCell#709', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_1047', 'r_ohm_per_km': '0.05', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '333', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '181': {'typ': 'Line', 'name': 'mxCell_711', 'id': 'mxCell#711', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_1125', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '182': {'typ': 'Bus70', 'name': 'mxCell_713', 'id': 'mxCell#713', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '183': {'typ': 'Line', 'name': 'mxCell_719', 'id': 'mxCell#719', 'busFrom': 'mxCell_721', 'busTo': 'mxCell_694', 'r_ohm_per_km': '0.01', 'x_ohm_per_km': '0.1', 'length_km': '1', 'c_nf_per_km': '256', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '1.15', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '184': {'typ': 'Bus71', 'name': 'mxCell_721', 'id': 'mxCell#721', 'vn_kv': '275', 'userFriendlyName': 'Bus'}, '185': {'typ': 'Bus72', 'name': 'mxCell_725', 'id': 'mxCell#725', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '186': {'typ': 'Bus73', 'name': 'mxCell_728', 'id': 'mxCell#728', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '187': {'typ': 'Static Generator', 'name': 'mxCell_733', 'id': 'mxCell#733', 'bus': 'mxCell_740', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '188': {'typ': 'Line', 'name': 'mxCell_735', 'id': 'mxCell#735', 'busFrom': 'mxCell_725', 'busTo': 'mxCell_737', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '189': {'typ': 'Bus74', 'name': 'mxCell_737', 'id': 'mxCell#737', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '190': {'typ': 'Bus75', 'name': 'mxCell_740', 'id': 'mxCell#740', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '191': {'typ': 'Transformer', 'name': 'mxCell_746', 'id': 'mxCell#746', 'busFrom': 'mxCell_725', 'busTo': 'mxCell_728', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '192': {'typ': 'Transformer', 'name': 'mxCell_750', 'id': 'mxCell#750', 'busFrom': 'mxCell_737', 'busTo': 'mxCell_740', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '193': {'typ': 'Static Generator', 'name': 'mxCell_753', 'id': 'mxCell#753', 'bus': 'mxCell_760', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '194': {'typ': 'Line', 'name': 'mxCell_755', 'id': 'mxCell#755', 'busFrom': 'mxCell_737', 'busTo': 'mxCell_757', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '195': {'typ': 'Bus76', 'name': 'mxCell_757', 'id': 'mxCell#757', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '196': {'typ': 'Bus77', 'name': 'mxCell_760', 'id': 'mxCell#760', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '197': {'typ': 'Transformer', 'name': 'mxCell_766', 'id': 'mxCell#766', 'busFrom': 'mxCell_757', 'busTo': 'mxCell_760', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '198': {'typ': 'Static Generator', 'name': 'mxCell_769', 'id': 'mxCell#769', 'bus': 'mxCell_776', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '199': {'typ': 'Line', 'name': 'mxCell_771', 'id': 'mxCell#771', 'busFrom': 'mxCell_757', 'busTo': 'mxCell_773', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '200': {'typ': 'Bus78', 'name': 'mxCell_773', 'id': 'mxCell#773', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '201': {'typ': 'Bus79', 'name': 'mxCell_776', 'id': 'mxCell#776', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '202': {'typ': 'Transformer', 'name': 'mxCell_782', 'id': 'mxCell#782', 'busFrom': 'mxCell_773', 'busTo': 'mxCell_776', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '203': {'typ': 'Static Generator', 'name': 'mxCell_785', 'id': 'mxCell#785', 'bus': 'mxCell_792', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '204': {'typ': 'Line', 'name': 'mxCell_787', 'id': 'mxCell#787', 'busFrom': 'mxCell_773', 'busTo': 'mxCell_789', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '205': {'typ': 'Bus80', 'name': 'mxCell_789', 'id': 'mxCell#789', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '206': {'typ': 'Bus81', 'name': 'mxCell_792', 'id': 'mxCell#792', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '207': {'typ': 'Transformer', 'name': 'mxCell_798', 'id': 'mxCell#798', 'busFrom': 'mxCell_789', 'busTo': 'mxCell_792', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '208': {'typ': 'Static Generator', 'name': 'mxCell_801', 'id': 'mxCell#801', 'bus': 'mxCell_806', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '209': {'typ': 'Bus82', 'name': 'mxCell_803', 'id': 'mxCell#803', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '210': {'typ': 'Bus83', 'name': 'mxCell_806', 'id': 'mxCell#806', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '211': {'typ': 'Static Generator', 'name': 'mxCell_811', 'id': 'mxCell#811', 'bus': 'mxCell_818', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '212': {'typ': 'Line', 'name': 'mxCell_813', 'id': 'mxCell#813', 'busFrom': 'mxCell_803', 'busTo': 'mxCell_815', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '213': {'typ': 'Bus84', 'name': 'mxCell_815', 'id': 'mxCell#815', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '214': {'typ': 'Bus85', 'name': 'mxCell_818', 'id': 'mxCell#818', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '215': {'typ': 'Transformer', 'name': 'mxCell_824', 'id': 'mxCell#824', 'busFrom': 'mxCell_803', 'busTo': 'mxCell_806', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '216': {'typ': 'Transformer', 'name': 'mxCell_828', 'id': 'mxCell#828', 'busFrom': 'mxCell_815', 'busTo': 'mxCell_818', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '217': {'typ': 'Static Generator', 'name': 'mxCell_831', 'id': 'mxCell#831', 'bus': 'mxCell_838', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '218': {'typ': 'Line', 'name': 'mxCell_833', 'id': 'mxCell#833', 'busFrom': 'mxCell_815', 'busTo': 'mxCell_835', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '219': {'typ': 'Bus86', 'name': 'mxCell_835', 'id': 'mxCell#835', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '220': {'typ': 'Bus87', 'name': 'mxCell_838', 'id': 'mxCell#838', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '221': {'typ': 'Transformer', 'name': 'mxCell_844', 'id': 'mxCell#844', 'busFrom': 'mxCell_835', 'busTo': 'mxCell_838', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '222': {'typ': 'Static Generator', 'name': 'mxCell_847', 'id': 'mxCell#847', 'bus': 'mxCell_854', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '223': {'typ': 'Line', 'name': 'mxCell_849', 'id': 'mxCell#849', 'busFrom': 'mxCell_835', 'busTo': 'mxCell_851', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '224': {'typ': 'Bus88', 'name': 'mxCell_851', 'id': 'mxCell#851', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '225': {'typ': 'Bus89', 'name': 'mxCell_854', 'id': 'mxCell#854', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '226': {'typ': 'Transformer', 'name': 'mxCell_860', 'id': 'mxCell#860', 'busFrom': 'mxCell_851', 'busTo': 'mxCell_854', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '227': {'typ': 'Static Generator', 'name': 'mxCell_863', 'id': 'mxCell#863', 'bus': 'mxCell_870', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '228': {'typ': 'Line', 'name': 'mxCell_865', 'id': 'mxCell#865', 'busFrom': 'mxCell_851', 'busTo': 'mxCell_867', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '229': {'typ': 'Bus90', 'name': 'mxCell_867', 'id': 'mxCell#867', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '230': {'typ': 'Bus91', 'name': 'mxCell_870', 'id': 'mxCell#870', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '231': {'typ': 'Transformer', 'name': 'mxCell_876', 'id': 'mxCell#876', 'busFrom': 'mxCell_867', 'busTo': 'mxCell_870', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '232': {'typ': 'Transformer', 'name': 'mxCell_880', 'id': 'mxCell#880', 'busFrom': 'mxCell_148', 'busTo': 'mxCell_689', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '233': {'typ': 'Transformer', 'name': 'mxCell_884', 'id': 'mxCell#884', 'busFrom': 'mxCell_697', 'busTo': 'mxCell_713', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '234': {'typ': 'Line', 'name': 'mxCell_885', 'id': 'mxCell#885', 'busFrom': 'mxCell_713', 'busTo': 'mxCell_891', 'r_ohm_per_km': '0.036', 'x_ohm_per_km': '0.106', 'length_km': '1', 'c_nf_per_km': '343', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.735', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '235': {'typ': 'Static Generator', 'name': 'mxCell_889', 'id': 'mxCell#889', 'bus': 'mxCell_894', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '236': {'typ': 'Bus92', 'name': 'mxCell_891', 'id': 'mxCell#891', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '237': {'typ': 'Bus93', 'name': 'mxCell_894', 'id': 'mxCell#894', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '238': {'typ': 'Static Generator', 'name': 'mxCell_899', 'id': 'mxCell#899', 'bus': 'mxCell_906', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '239': {'typ': 'Line', 'name': 'mxCell_901', 'id': 'mxCell#901', 'busFrom': 'mxCell_891', 'busTo': 'mxCell_903', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '240': {'typ': 'Bus94', 'name': 'mxCell_903', 'id': 'mxCell#903', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '241': {'typ': 'Bus95', 'name': 'mxCell_906', 'id': 'mxCell#906', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '242': {'typ': 'Transformer', 'name': 'mxCell_912', 'id': 'mxCell#912', 'busFrom': 'mxCell_891', 'busTo': 'mxCell_894', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '243': {'typ': 'Transformer', 'name': 'mxCell_916', 'id': 'mxCell#916', 'busFrom': 'mxCell_903', 'busTo': 'mxCell_906', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '244': {'typ': 'Static Generator', 'name': 'mxCell_919', 'id': 'mxCell#919', 'bus': 'mxCell_926', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '245': {'typ': 'Line', 'name': 'mxCell_921', 'id': 'mxCell#921', 'busFrom': 'mxCell_903', 'busTo': 'mxCell_923', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '246': {'typ': 'Bus96', 'name': 'mxCell_923', 'id': 'mxCell#923', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '247': {'typ': 'Bus97', 'name': 'mxCell_926', 'id': 'mxCell#926', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '248': {'typ': 'Transformer', 'name': 'mxCell_932', 'id': 'mxCell#932', 'busFrom': 'mxCell_923', 'busTo': 'mxCell_926', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '249': {'typ': 'Static Generator', 'name': 'mxCell_935', 'id': 'mxCell#935', 'bus': 'mxCell_942', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '250': {'typ': 'Line', 'name': 'mxCell_937', 'id': 'mxCell#937', 'busFrom': 'mxCell_923', 'busTo': 'mxCell_939', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '251': {'typ': 'Bus98', 'name': 'mxCell_939', 'id': 'mxCell#939', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '252': {'typ': 'Bus99', 'name': 'mxCell_942', 'id': 'mxCell#942', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '253': {'typ': 'Transformer', 'name': 'mxCell_948', 'id': 'mxCell#948', 'busFrom': 'mxCell_939', 'busTo': 'mxCell_942', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '254': {'typ': 'Static Generator', 'name': 'mxCell_951', 'id': 'mxCell#951', 'bus': 'mxCell_958', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '255': {'typ': 'Line', 'name': 'mxCell_953', 'id': 'mxCell#953', 'busFrom': 'mxCell_939', 'busTo': 'mxCell_955', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '256': {'typ': 'Bus100', 'name': 'mxCell_955', 'id': 'mxCell#955', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '257': {'typ': 'Bus101', 'name': 'mxCell_958', 'id': 'mxCell#958', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '258': {'typ': 'Transformer', 'name': 'mxCell_964', 'id': 'mxCell#964', 'busFrom': 'mxCell_955', 'busTo': 'mxCell_958', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '259': {'typ': 'Static Generator', 'name': 'mxCell_967', 'id': 'mxCell#967', 'bus': 'mxCell_972', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '260': {'typ': 'Bus102', 'name': 'mxCell_969', 'id': 'mxCell#969', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '261': {'typ': 'Bus103', 'name': 'mxCell_972', 'id': 'mxCell#972', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '262': {'typ': 'Static Generator', 'name': 'mxCell_977', 'id': 'mxCell#977', 'bus': 'mxCell_984', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '263': {'typ': 'Line', 'name': 'mxCell_979', 'id': 'mxCell#979', 'busFrom': 'mxCell_969', 'busTo': 'mxCell_981', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '264': {'typ': 'Bus104', 'name': 'mxCell_981', 'id': 'mxCell#981', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '265': {'typ': 'Bus105', 'name': 'mxCell_984', 'id': 'mxCell#984', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '266': {'typ': 'Transformer', 'name': 'mxCell_990', 'id': 'mxCell#990', 'busFrom': 'mxCell_969', 'busTo': 'mxCell_972', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '267': {'typ': 'Transformer', 'name': 'mxCell_994', 'id': 'mxCell#994', 'busFrom': 'mxCell_981', 'busTo': 'mxCell_984', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '268': {'typ': 'Static Generator', 'name': 'mxCell_997', 'id': 'mxCell#997', 'bus': 'mxCell_1004', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '269': {'typ': 'Line', 'name': 'mxCell_999', 'id': 'mxCell#999', 'busFrom': 'mxCell_981', 'busTo': 'mxCell_1001', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '270': {'typ': 'Bus106', 'name': 'mxCell_1001', 'id': 'mxCell#1001', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '271': {'typ': 'Bus107', 'name': 'mxCell_1004', 'id': 'mxCell#1004', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '272': {'typ': 'Transformer', 'name': 'mxCell_1010', 'id': 'mxCell#1010', 'busFrom': 'mxCell_1001', 'busTo': 'mxCell_1004', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '273': {'typ': 'Static Generator', 'name': 'mxCell_1013', 'id': 'mxCell#1013', 'bus': 'mxCell_1020', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '274': {'typ': 'Line', 'name': 'mxCell_1015', 'id': 'mxCell#1015', 'busFrom': 'mxCell_1001', 'busTo': 'mxCell_1017', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '275': {'typ': 'Bus108', 'name': 'mxCell_1017', 'id': 'mxCell#1017', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '276': {'typ': 'Bus109', 'name': 'mxCell_1020', 'id': 'mxCell#1020', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '277': {'typ': 'Transformer', 'name': 'mxCell_1026', 'id': 'mxCell#1026', 'busFrom': 'mxCell_1017', 'busTo': 'mxCell_1020', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '278': {'typ': 'Static Generator', 'name': 'mxCell_1029', 'id': 'mxCell#1029', 'bus': 'mxCell_1036', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '279': {'typ': 'Line', 'name': 'mxCell_1031', 'id': 'mxCell#1031', 'busFrom': 'mxCell_1017', 'busTo': 'mxCell_1033', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '280': {'typ': 'Bus110', 'name': 'mxCell_1033', 'id': 'mxCell#1033', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '281': {'typ': 'Bus111', 'name': 'mxCell_1036', 'id': 'mxCell#1036', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '282': {'typ': 'Transformer', 'name': 'mxCell_1042', 'id': 'mxCell#1042', 'busFrom': 'mxCell_1033', 'busTo': 'mxCell_1036', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '283': {'typ': 'Static Generator', 'name': 'mxCell_1045', 'id': 'mxCell#1045', 'bus': 'mxCell_1050', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '284': {'typ': 'Bus112', 'name': 'mxCell_1047', 'id': 'mxCell#1047', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '285': {'typ': 'Bus113', 'name': 'mxCell_1050', 'id': 'mxCell#1050', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '286': {'typ': 'Static Generator', 'name': 'mxCell_1055', 'id': 'mxCell#1055', 'bus': 'mxCell_1062', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '287': {'typ': 'Line', 'name': 'mxCell_1057', 'id': 'mxCell#1057', 'busFrom': 'mxCell_1047', 'busTo': 'mxCell_1059', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '288': {'typ': 'Bus114', 'name': 'mxCell_1059', 'id': 'mxCell#1059', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '289': {'typ': 'Bus115', 'name': 'mxCell_1062', 'id': 'mxCell#1062', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '290': {'typ': 'Transformer', 'name': 'mxCell_1068', 'id': 'mxCell#1068', 'busFrom': 'mxCell_1047', 'busTo': 'mxCell_1050', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '291': {'typ': 'Transformer', 'name': 'mxCell_1072', 'id': 'mxCell#1072', 'busFrom': 'mxCell_1059', 'busTo': 'mxCell_1062', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '292': {'typ': 'Static Generator', 'name': 'mxCell_1075', 'id': 'mxCell#1075', 'bus': 'mxCell_1082', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '293': {'typ': 'Line', 'name': 'mxCell_1077', 'id': 'mxCell#1077', 'busFrom': 'mxCell_1059', 'busTo': 'mxCell_1079', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '294': {'typ': 'Bus116', 'name': 'mxCell_1079', 'id': 'mxCell#1079', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '295': {'typ': 'Bus117', 'name': 'mxCell_1082', 'id': 'mxCell#1082', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '296': {'typ': 'Transformer', 'name': 'mxCell_1088', 'id': 'mxCell#1088', 'busFrom': 'mxCell_1079', 'busTo': 'mxCell_1082', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '297': {'typ': 'Static Generator', 'name': 'mxCell_1091', 'id': 'mxCell#1091', 'bus': 'mxCell_1098', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '298': {'typ': 'Line', 'name': 'mxCell_1093', 'id': 'mxCell#1093', 'busFrom': 'mxCell_1079', 'busTo': 'mxCell_1095', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '299': {'typ': 'Bus118', 'name': 'mxCell_1095', 'id': 'mxCell#1095', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '300': {'typ': 'Bus119', 'name': 'mxCell_1098', 'id': 'mxCell#1098', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '301': {'typ': 'Transformer', 'name': 'mxCell_1104', 'id': 'mxCell#1104', 'busFrom': 'mxCell_1095', 'busTo': 'mxCell_1098', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '302': {'typ': 'Static Generator', 'name': 'mxCell_1107', 'id': 'mxCell#1107', 'bus': 'mxCell_1114', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '303': {'typ': 'Line', 'name': 'mxCell_1109', 'id': 'mxCell#1109', 'busFrom': 'mxCell_1095', 'busTo': 'mxCell_1111', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '304': {'typ': 'Bus120', 'name': 'mxCell_1111', 'id': 'mxCell#1111', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '305': {'typ': 'Bus121', 'name': 'mxCell_1114', 'id': 'mxCell#1114', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '306': {'typ': 'Transformer', 'name': 'mxCell_1120', 'id': 'mxCell#1120', 'busFrom': 'mxCell_1111', 'busTo': 'mxCell_1114', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '307': {'typ': 'Static Generator', 'name': 'mxCell_1123', 'id': 'mxCell#1123', 'bus': 'mxCell_1128', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '308': {'typ': 'Bus122', 'name': 'mxCell_1125', 'id': 'mxCell#1125', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '309': {'typ': 'Bus123', 'name': 'mxCell_1128', 'id': 'mxCell#1128', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '310': {'typ': 'Static Generator', 'name': 'mxCell_1133', 'id': 'mxCell#1133', 'bus': 'mxCell_1140', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '311': {'typ': 'Line', 'name': 'mxCell_1135', 'id': 'mxCell#1135', 'busFrom': 'mxCell_1125', 'busTo': 'mxCell_1137', 'r_ohm_per_km': '0.032', 'x_ohm_per_km': '0.139', 'length_km': '1', 'c_nf_per_km': '159', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.832', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '312': {'typ': 'Bus124', 'name': 'mxCell_1137', 'id': 'mxCell#1137', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '313': {'typ': 'Bus125', 'name': 'mxCell_1140', 'id': 'mxCell#1140', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '314': {'typ': 'Transformer', 'name': 'mxCell_1146', 'id': 'mxCell#1146', 'busFrom': 'mxCell_1125', 'busTo': 'mxCell_1128', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '315': {'typ': 'Transformer', 'name': 'mxCell_1150', 'id': 'mxCell#1150', 'busFrom': 'mxCell_1137', 'busTo': 'mxCell_1140', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '316': {'typ': 'Static Generator', 'name': 'mxCell_1153', 'id': 'mxCell#1153', 'bus': 'mxCell_1160', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '317': {'typ': 'Line', 'name': 'mxCell_1155', 'id': 'mxCell#1155', 'busFrom': 'mxCell_1137', 'busTo': 'mxCell_1157', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '318': {'typ': 'Bus126', 'name': 'mxCell_1157', 'id': 'mxCell#1157', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '319': {'typ': 'Bus127', 'name': 'mxCell_1160', 'id': 'mxCell#1160', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '320': {'typ': 'Transformer', 'name': 'mxCell_1166', 'id': 'mxCell#1166', 'busFrom': 'mxCell_1157', 'busTo': 'mxCell_1160', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '321': {'typ': 'Static Generator', 'name': 'mxCell_1169', 'id': 'mxCell#1169', 'bus': 'mxCell_1176', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '322': {'typ': 'Line', 'name': 'mxCell_1171', 'id': 'mxCell#1171', 'busFrom': 'mxCell_1157', 'busTo': 'mxCell_1173', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '323': {'typ': 'Bus128', 'name': 'mxCell_1173', 'id': 'mxCell#1173', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '324': {'typ': 'Bus129', 'name': 'mxCell_1176', 'id': 'mxCell#1176', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '325': {'typ': 'Transformer', 'name': 'mxCell_1182', 'id': 'mxCell#1182', 'busFrom': 'mxCell_1173', 'busTo': 'mxCell_1176', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '326': {'typ': 'Static Generator', 'name': 'mxCell_1185', 'id': 'mxCell#1185', 'bus': 'mxCell_1192', 'p_mw': '15', 'q_mvar': '3', 'sn_mva': '0', 'scaling': '1', 'type': 'wye', 'max_p_mw': 1, 'min_p_mw': 0, 'max_q_mvar': 1, 'min_q_mvar': -1, 'vm_pu': 1, 'va_degree': 0, 'k': '0', 'rx': '0', 'generator_type': 'async', 'lrc_pu': '0', 'max_ik_ka': '0', 'kappa': '0', 'current_source': 'true', 'in_service': True}, '327': {'typ': 'Line', 'name': 'mxCell_1187', 'id': 'mxCell#1187', 'busFrom': 'mxCell_1173', 'busTo': 'mxCell_1189', 'r_ohm_per_km': '0.13', 'x_ohm_per_km': '0.122', 'length_km': '1', 'c_nf_per_km': '228', 'parallel': '1', 'df': '1', 'g_us_per_km': '0', 'max_i_ka': '0.465', 'type': 'cs', 'in_service': True, 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0.1', 'c0_nf_per_km': '0.1', 'endtemp_degree': '250'}, '328': {'typ': 'Bus130', 'name': 'mxCell_1189', 'id': 'mxCell#1189', 'vn_kv': '66', 'userFriendlyName': 'Bus'}, '329': {'typ': 'Bus131', 'name': 'mxCell_1192', 'id': 'mxCell#1192', 'vn_kv': '0.69', 'userFriendlyName': 'Bus'}, '330': {'typ': 'Transformer', 'name': 'mxCell_1198', 'id': 'mxCell#1198', 'busFrom': 'mxCell_1189', 'busTo': 'mxCell_1192', 'sn_mva': '16.8', 'vk_percent': '8', 'vkr_percent': '1', 'vn_hv_kv': '66', 'vn_lv_kv': '0.69', 'pfe_kw': '76', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-6', 'tap_max': '6', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '6', 'tap_phase_shifter': 'false', 'in_service': True}, '331': {'typ': 'Transformer', 'name': 'mxCell_1202', 'id': 'mxCell#1202', 'busFrom': 'mxCell_697', 'busTo': 'mxCell_713', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '275', 'vn_lv_kv': '66', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '332': {'typ': 'Transformer', 'name': 'mxCell_1206', 'id': 'mxCell#1206', 'busFrom': 'mxCell_148', 'busTo': 'mxCell_689', 'sn_mva': '300', 'vk_percent': '12', 'vkr_percent': '1', 'vn_hv_kv': '400', 'vn_lv_kv': '275', 'pfe_kw': '100', 'i0_percent': '1', 'shift_degree': '330', 'tap_side': 'hv', 'tap_neutral': '0', 'tap_min': '-10', 'tap_max': '10', 'tap_step_percent': '1.5', 'tap_step_degree': '0', 'tap_pos': '0', 'tap_phase_shifter': 'false', 'in_service': True}, '333': {'typ': 'Shunt Reactor', 'name': 'mxCell_1209', 'id': 'mxCell#1209', 'bus': 'mxCell_697', 'p_mw': '1', 'q_mvar': '200', 'sn_mva': 1, 'scaling': 1, 'type': 'const_q', 'step': '1', 'step_degree': 0, 'step_min': 0, 'step_max': 1, 'step_pos': 1, 'step_phase_shifter': False, 'in_service': True}
}



In [2]:
# Load flow from backend modules (same logic as app.py)
import pandapower as pp
import pandapower_electrisim
from opendss_electrisim import powerflow, shortcircuit
from pandapower_electrisim import powerflow as powerflow_pandapower, shortcircuit as shortcircuit_pandapower


In [3]:
# 1) Run PandaPower load flow with in_data_pandapower
pf_panda_key = next((k for k in in_data_pandapower if isinstance(in_data_pandapower[k], dict) and 'PowerFlowPandaPower' in in_data_pandapower[k].get('typ', '')), None)
res_pandapower = None
if pf_panda_key is not None:
    p = in_data_pandapower[pf_panda_key]
    frequency = eval(p['frequency'])
    net = pp.create_empty_network(f_hz=frequency)
    Busbars = pandapower_electrisim.create_busbars(in_data_pandapower, net)
    pandapower_electrisim.create_other_elements(in_data_pandapower, net, pf_panda_key, Busbars)
    # Force export of pandapower Python code so we can log native element-creation calls
    export_python = True
    response_panda = powerflow_pandapower(net, p['algorithm'], p['calculate_voltage_angles'], p['initialization'],
                                         export_python, in_data_pandapower, Busbars, run_control=p.get('run_control', False))
    if 'error' in json.loads(response_panda):
        print("PandaPower error:", json.loads(response_panda)['error'])
    else:
        res_pandapower = json.loads(response_panda)
        print("PandaPower load flow: OK")
        # Log pandapower element-creation code (native pp.create_* calls)
        pp_code = res_pandapower.get('pandapower_python')
        if pp_code:
            print("\n" + "=" * 80)
            print("PANDAPOWER NETWORK CREATION (native pandapower API)")
            print("=" * 80)
            if isinstance(pp_code, list):
                for line in pp_code:
                    print(f"[PandaPower] {line}")
            elif isinstance(pp_code, str):
                for line in pp_code.splitlines():
                    print(f"[PandaPower] {line}")
else:
    print("PandaPower: No PowerFlowPandaPower Parameters in in_data_pandapower")

# 2) Run OpenDSS load flow with in_data_opendss
pf_open_key = next((k for k in in_data_opendss if isinstance(in_data_opendss[k], dict) and 'PowerFlowOpenDss' in in_data_opendss[k].get('typ', '')), None)
res_opendss = None
if pf_open_key is not None:
    p = in_data_opendss[pf_open_key]
    response_open = powerflow(in_data_opendss, eval(p['frequency']), p.get('mode', 'Snapshot'), p.get('algorithm', 'Normal'),
                             p.get('loadmodel', 'Powerflow'), int(p.get('maxIterations', 100)), float(p.get('tolerance', 0.0001)),
                             p.get('controlmode', 'Static'), p.get('exportCommands', False))
    if 'error' in json.loads(response_open):
        print("OpenDSS error:", json.loads(response_open)['error'])
    else:
        res_opendss = json.loads(response_open)
        print("OpenDSS load flow: OK")
else:
    print("OpenDSS: No PowerFlowOpenDss Parameters in in_data_opendss")

# 3) Compare results (by bus name)
def by_name(items, key='name'):
    return {item.get(key): item for item in (items or []) if item.get(key)}

if res_pandapower is not None and res_opendss is not None:
    buses_pp = by_name(res_pandapower.get('busbars', []))
    buses_od = by_name(res_opendss.get('busbars', []))
    all_names = sorted(set(buses_pp) | set(buses_od))
    tol = 1e-4  # relative tolerance for "match"

    print("\n" + "=" * 80)
    print("COMPARISON: PandaPower vs OpenDSS (busbars)")
    print("=" * 80)
    for name in all_names:
        b_pp = buses_pp.get(name, {})
        b_od = buses_od.get(name, {})
        vm_pp = b_pp.get('vm_pu')
        vm_od = b_od.get('vm_pu')
        va_pp = b_pp.get('va_degree')
        va_od = b_od.get('va_degree')
        p_pp = b_pp.get('p_mw')
        p_od = b_od.get('p_mw')
        q_pp = b_pp.get('q_mvar')
        q_od = b_od.get('q_mvar')
        match = (vm_pp is not None and vm_od is not None and abs((vm_pp or 0) - (vm_od or 0)) < tol and
                 abs((va_pp or 0) - (va_od or 0)) < tol and abs((p_pp or 0) - (p_od or 0)) < tol and
                 abs((q_pp or 0) - (q_od or 0)) < tol)
        status = "OK" if match else "DIFF"
        print(f"  {name}: {status}  |  PandaPower: vm_pu={vm_pp}, va_deg={va_pp}, P={p_pp}, Q={q_pp}  |  OpenDSS: vm_pu={vm_od}, va_deg={va_od}, P={p_od}, Q={q_od}")

    # Summary
    ext_pp = by_name(res_pandapower.get('externalgrids', []))
    ext_od = by_name(res_opendss.get('externalgrids', []))
    # PandaPower puts your mxCell_173 in "staticgenerators", while OpenDSS reports it as a generator
    gen_pp = by_name((res_pandapower.get('generators', []) or []) + (res_pandapower.get('staticgenerators', []) or []))
    gen_od = by_name(res_opendss.get('generators', []))
    print("\n--- External grids ---")
    for name in sorted(set(ext_pp) | set(ext_od)):
        e_pp, e_od = ext_pp.get(name, {}), ext_od.get(name, {})
        print(f"  {name}:  PandaPower P={e_pp.get('p_mw')} Q={e_pp.get('q_mvar')}  |  OpenDSS P={e_od.get('p_mw')} Q={e_od.get('q_mvar')}")
    print("--- Generators / Static generators ---")
    for name in sorted(set(gen_pp) | set(gen_od)):
        g_pp, g_od = gen_pp.get(name, {}), gen_od.get(name, {})
        print(f"  {name}:  PandaPower P={g_pp.get('p_mw')} Q={g_pp.get('q_mvar')}  |  OpenDSS P={g_od.get('p_mw')} Q={g_od.get('q_mvar')}")

c:\Users\DELL\.vscode\appElectrisimBackend\appElectrisimBackend\.venv\lib\site-packages\pandapower\create\_utils.py:328: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  net[table].at[index, col] = val
c:\Users\DELL\.vscode\appElectrisimBackend\appElectrisimBackend\.venv\lib\site-packages\pandapower\create\_utils.py:328: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  net[table].at[index, col] = val
c:\Users\DELL\.vscode\appElectrisimBackend\appElectrisimBackend\.venv\lib\site-packages\pandapower\create\_utils.py:328: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value

⚡ Running power flow WITHOUT controllers (run_control=False)
Response to FRONTEND CORRECT
PandaPower load flow: OK

PANDAPOWER NETWORK CREATION (native pandapower API)
[PandaPower] # Pandapower Network Model
[PandaPower] # Auto-generated code to recreate the network
[PandaPower] 
[PandaPower] import pandapower as pp
[PandaPower] 
[PandaPower] # Create empty network with 50 Hz
[PandaPower] net = pp.create_empty_network(f_hz=50)
[PandaPower] 
[PandaPower] # Create buses
[PandaPower] bus_0 = pp.create_bus(net, vn_kv=400.0, name='mxCell_145')
[PandaPower] bus_1 = pp.create_bus(net, vn_kv=400.0, name='mxCell_148')
[PandaPower] bus_2 = pp.create_bus(net, vn_kv=275.0, name='mxCell_157')
[PandaPower] bus_3 = pp.create_bus(net, vn_kv=275.0, name='mxCell_162')
[PandaPower] bus_4 = pp.create_bus(net, vn_kv=275.0, name='mxCell_165')
[PandaPower] bus_5 = pp.create_bus(net, vn_kv=66.0, name='mxCell_181')
[PandaPower] bus_6 = pp.create_bus(net, vn_kv=275.0, name='mxCell_190')
[PandaPower] bus_7 = pp.